In [ ]:
# import packages

# packages for data preparation and plotting
import pandas as pd
import os
import csv
import netCDF4 as nc
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import table
import holidays
import cftime
from netCDF4 import Dataset
import datetime as dt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import rcParams

# packages for modeling and evaluation
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, GradientBoostingRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import Huber

from scipy.stats import kendalltau, spearmanr

# other
import warnings

In [ ]:
# Data wrangling

In [ ]:
# Load, inspect and clean the flight delay dataset

In [ ]:
# loading and combining the flight data

warnings.filterwarnings("ignore")

flight_data = pd.DataFrame()

data_folder = "./flight_data"
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

for csv_file in csv_files:
    file_path = os.path.join(data_folder, csv_file)
    df = pd.read_csv(file_path)
    flight_data = flight_data.append(df, ignore_index=True)

flight_data

In [ ]:
# Filter for JFK airport

df_1 = flight_data[flight_data['ORIGIN'] == 'JFK']
df_1

In [ ]:
# Sorting by date

df_1['FL_DATE'] = pd.to_datetime(df_1['FL_DATE'])

df_1 = df_1.sort_values(by='FL_DATE')

df_1 = df_1.reset_index(drop=True)

df_1

In [ ]:
# Inspecting the data
# Inspecting null values

In [ ]:
df_1.isnull().sum()

In [ ]:
missing_values_rows = df_1[df_1['AIR_TIME'].isnull()]

# Display the rows with missing values in 'arr_flights'
print(missing_values_rows)

In [ ]:
df_1.dropna(subset=['AIR_TIME'], inplace=True)

In [ ]:
# replace NaN values in a column with 0
df_1['DEP_DELAY_NEW'].fillna(0, inplace=True)

In [ ]:
# replace NaN values in a column with 0
df_1['WEATHER_DELAY'].fillna(0, inplace=True)

In [ ]:
# replace NaN values in a column with 0
df_1['NAS_DELAY'].fillna(0, inplace=True)

In [ ]:
df_1

In [ ]:
df_1.dtypes

In [ ]:
df_1.describe()

In [ ]:
# Create a binary variable Delayed. Is 0 if on-time and is 1 if delayed.

df_1['Delayed'] = np.where((df_1['DEP_DELAY_NEW'] > 0) | (df_1['CANCELLED'] > 0), 1, 0)
df_1

In [ ]:
# Create a binary variable Weather_Delayed. Is 0 if on-time and is 1 if delayed because of weather.

df_1['Weather_Delayed'] = np.where((df_1['WEATHER_DELAY'] > 0) | (df_1['NAS_DELAY'] > 0), 1, 0)
df_1

In [ ]:
# Add a column to classify whether the date is a weekday or weekend

# Create a custom function to classify the days
def classify_day_type(date):
    if date.weekday() < 5:  # 0-4 correspond to Monday to Friday
        return 'weekday'
    else:
        return 'weekend'

# Apply the function to create the "Day_Type" column
df_1['Day_Type'] = df_1['FL_DATE'].apply(classify_day_type)
df_1

In [ ]:
df_1

In [ ]:
# load, inspect and clean the ERA5 climate data

In [ ]:
# wind data

In [ ]:
# inspect dataset
wind_ds = nc.Dataset('./ERA5_data/ERA5_10mwindgust.nc', 'r')
wind_ds

In [ ]:
# List dimensions
print("Dimensions:")
for dimname, dimobj in wind_ds.dimensions.items():
    print(f"{dimname}: {len(dimobj)}")

# List variables
print("\nVariables:")
for varname, varobj in wind_ds.variables.items():
    print(f"{varname}: {varobj.shape} - {varobj.dtype}")

# List global attributes
print("\nGlobal Attributes:")
for attrname in wind_ds.ncattrs():
    print(f"{attrname}: {getattr(wind_ds, attrname)}")


In [ ]:
# Load the netCDF dataset
wind_ds = nc.Dataset('./ERA5_data/ERA5_10mwindgust.nc', 'r')

# Extract time and wind gust values
time_values = wind_ds.variables['time'][:]
wind_gust_values = wind_ds.variables['fg10'][:, :, 0, 0]

# Filter data until August 31, 2023
end_date = pd.to_datetime("2023-08-31")
filtered_indices = time_values <= pd.Timestamp(end_date).timestamp()

# Convert time values to a human-readable format
hours = time_values[filtered_indices] % 24
dates = pd.to_datetime(time_values[filtered_indices], unit='h', origin='1900-01-01')
dates += pd.to_timedelta(hours, unit='h')

# Create a DataFrame
df = pd.DataFrame({'Date': dates, 'Wind Gust': wind_gust_values[filtered_indices, 0]})

# Group by date and calculate both the maximum and average wind gust values for each day
df_2 = df.groupby(df['Date'].dt.date)['Wind Gust'].agg(['max', 'mean']).reset_index()
df_2.columns = ['Date', 'Max Wind Gust', 'Average Wind Gust']

# Print the DataFrame with both maximum and average wind gust values per day
print(df_2)


In [ ]:
# Convert the 'Date' column to a datetime type (if it's not already)
df_2['Date'] = pd.to_datetime(df_2['Date'])

# Filter the DataFrame to include only rows until 2023-08-31
df_2 = df_2[df_2['Date'] <= '2023-08-31']
df_2

In [ ]:
# Merge the dataframes on the 'FL_DATE' and 'Date' columns
merged_df = df_1.merge(df_2, left_on='FL_DATE', right_on='Date', how='left')

# Drop the 'Date' column 
merged_df = merged_df.drop(columns=['Date'])

# Now, merged_df contains the Max Wind Gust and Average Wind Gust columns from df_2

In [ ]:
merged_df

In [ ]:
# max total precipitation

In [ ]:
# inspect dataset
prec_ds = nc.Dataset('./ERA5_data/ERA5_maxtotalprec.nc', 'r')
prec_ds

In [ ]:
# List dimensions
print("Dimensions:")
for dimname, dimobj in prec_ds.dimensions.items():
    print(f"{dimname}: {len(dimobj)}")

# List variables
print("\nVariables:")
for varname, varobj in prec_ds.variables.items():
    print(f"{varname}: {varobj.shape} - {varobj.dtype}")

# List global attributes
print("\nGlobal Attributes:")
for attrname in prec_ds.ncattrs():
    print(f"{attrname}: {getattr(prec_ds, attrname)}")

In [ ]:
# Load the netCDF dataset
prec_ds = nc.Dataset('./ERA5_data/ERA5_maxtotalprec.nc', 'r')

# Extract time and wind gust values
time_values = prec_ds.variables['time'][:]
prec_values = prec_ds.variables['mxtpr'][:, :, 0, 0]

# Filter data until August 31, 2023
end_date = pd.to_datetime("2023-08-31")
filtered_indices = time_values <= pd.Timestamp(end_date).timestamp()

# Convert time values to a human-readable format (you can adjust this as needed)
hours = time_values[filtered_indices] % 24
dates = pd.to_datetime(time_values[filtered_indices], unit='h', origin='1900-01-01')
dates += pd.to_timedelta(hours, unit='h')

# Create a DataFrame
df_3 = pd.DataFrame({'Date': dates, 'Precipitation': prec_values[filtered_indices, 0]})

# Group by date and calculate both the maximum and average wind gust values for each day
df_3 = df_3.groupby(df_3['Date'].dt.date)['Precipitation'].agg(['max', 'mean']).reset_index()
df_3.columns = ['Date', 'Max Precipitation', 'Average Precipitation']

# Print the DataFrame with both maximum and average wind gust values per day
print(df_3)


In [ ]:
# Convert the 'Date' column to a datetime type (if it's not already)
df_3['Date'] = pd.to_datetime(df_3['Date'])

# Filter the DataFrame to include only rows until 2023-08-31
df_3 = df_3[df_3['Date'] <= '2023-08-31']
df_3

In [ ]:
# Merge the dataframes on the 'FL_DATE' and 'Date' columns

df_3['Date'] = pd.to_datetime(df_3['Date'])
merged_df_2 = merged_df.merge(df_3, left_on='FL_DATE', right_on='Date', how='left')

# Drop the 'Date' column from the merged dataframe if you don't need it
merged_df_2 = merged_df_2.drop(columns=['Date'])

# Now, merged_df contains the Max Wind Gust and Average Wind Gust columns from df_2
merged_df_2

In [ ]:
# max temperature

In [ ]:
# inspect dataset
maxtemp_ds = nc.Dataset('./ERA5_data/ERA5_max2mtemp.nc', 'r')
maxtemp_ds

In [ ]:
# List dimensions
print("Dimensions:")
for dimname, dimobj in maxtemp_ds.dimensions.items():
    print(f"{dimname}: {len(dimobj)}")

# List variables
print("\nVariables:")
for varname, varobj in maxtemp_ds.variables.items():
    print(f"{varname}: {varobj.shape} - {varobj.dtype}")

# List global attributes
print("\nGlobal Attributes:")
for attrname in maxtemp_ds.ncattrs():
    print(f"{attrname}: {getattr(maxtemp_ds, attrname)}")


In [ ]:
# Load the netCDF dataset
maxtemp_ds = nc.Dataset('./ERA5_data/ERA5_max2mtemp.nc', 'r')

# Extract time and wind gust values
time_values = maxtemp_ds.variables['time'][:]
maxtemp_values = maxtemp_ds.variables['mx2t'][:, :, 0, 0]

# Filter data until August 31, 2023
end_date = pd.to_datetime("2023-08-31")
filtered_indices = time_values <= pd.Timestamp(end_date).timestamp()

# Convert time values to a human-readable format (you can adjust this as needed)
hours = time_values[filtered_indices] % 24
dates = pd.to_datetime(time_values[filtered_indices], unit='h', origin='1900-01-01')
dates += pd.to_timedelta(hours, unit='h')

# Create a DataFrame
df_4 = pd.DataFrame({'Date': dates, 'Max Temperature': maxtemp_values[filtered_indices, 0]})

# Group by date and calculate both the maximum and average wind gust values for each day
df_4 = df_4.groupby(df_4['Date'].dt.date)['Max Temperature'].agg(['max']).reset_index()
df_4.columns = ['Date', 'Max Temperature']

# Print the DataFrame with both maximum and average wind gust values per day
print(df_4)


In [ ]:
# Understand where NaN values start
first_nan_date = df_4[df_4['Max Temperature'].isna()]['Date'].iloc[0]

print("The NaN values start from:", first_nan_date)

In [ ]:
# Convert the 'Date' column to a datetime type (if it's not already)
df_4['Date'] = pd.to_datetime(df_4['Date'])

# Filter the DataFrame to include only rows until 2023-08-31
df_4 = df_4[df_4['Date'] <= '2023-07-31']
df_4

In [ ]:
# Convert the temperature values to °C
df_4['Max Temperature (°C)'] = df_4['Max Temperature'] - 273.15

In [ ]:
df_4

In [ ]:
# merge flight delay and temperature dataframes
import pandas as pd

merged_df_2['FL_DATE'] = pd.to_datetime(merged_df_2['FL_DATE'])  # Convert the 'FL_DATE' column to a datetime type
end_date = pd.to_datetime('2023-07-31')  # Define the end date

# Filter the DataFrame to keep only rows up to July 31, 2023
merged_df_2 = merged_df_2[merged_df_2['FL_DATE'] <= end_date]
merged_df_2

In [ ]:
# Merge the dataframes on the 'FL_DATE' and 'Date' columns
merged_df_3 = merged_df_2.merge(df_4, left_on='FL_DATE', right_on='Date', how='left')

# Drop the 'Date' column from the merged dataframe if you don't need it
merged_df_3 = merged_df_3.drop(columns=['Date'])

# Now, merged_df contains the Max Wind Gust and Average Wind Gust columns from df_2
merged_df_3

In [ ]:
# min temperature

In [ ]:
# inspect dataset
mintemp_ds = nc.Dataset('./ERA5_data/ERA5_min2mtemp.nc', 'r')
mintemp_ds

In [ ]:
# List dimensions
print("Dimensions:")
for dimname, dimobj in mintemp_ds.dimensions.items():
    print(f"{dimname}: {len(dimobj)}")

# List variables
print("\nVariables:")
for varname, varobj in mintemp_ds.variables.items():
    print(f"{varname}: {varobj.shape} - {varobj.dtype}")

# List global attributes
print("\nGlobal Attributes:")
for attrname in mintemp_ds.ncattrs():
    print(f"{attrname}: {getattr(mintemp_ds, attrname)}")


In [ ]:
# Load the netCDF dataset
mintemp_ds = nc.Dataset('./ERA5_data/ERA5_min2mtemp.nc', 'r')

# Extract time and wind gust values
time_values = mintemp_ds.variables['time'][:]
mintemp_values = mintemp_ds.variables['mn2t'][:, :, 0, 0]

# Filter data until August 31, 2023
end_date = pd.to_datetime("2023-08-31")
filtered_indices = time_values <= pd.Timestamp(end_date).timestamp()

# Convert time values to a human-readable format (you can adjust this as needed)
hours = time_values[filtered_indices] % 24
dates = pd.to_datetime(time_values[filtered_indices], unit='h', origin='1900-01-01')
dates += pd.to_timedelta(hours, unit='h')

# Create a DataFrame
df_5 = pd.DataFrame({'Date': dates, 'Min Temperature': mintemp_values[filtered_indices, 0]})

# Group by date and calculate both the maximum and average wind gust values for each day
df_5 = df_5.groupby(df_5['Date'].dt.date)['Min Temperature'].agg(['min']).reset_index()
df_5.columns = ['Date', 'Min Temperature']

# Print the DataFrame with both maximum and average wind gust values per day
print(df_5)


In [ ]:
# Convert the 'Date' column to a datetime type (if it's not already)
df_5['Date'] = pd.to_datetime(df_5['Date'])

# Filter the DataFrame to include only rows until 2023-08-31
df_5 = df_5[df_5['Date'] <= '2023-07-31']
df_5

In [ ]:
# Change temeprature values to °C
df_5['Min Temperature (°C)'] = df_5['Min Temperature'] - 273.15

In [ ]:
df_5

In [ ]:
# Merge the dataframes on the 'FL_DATE' and 'Date' columns
merged_df_4 = merged_df_3.merge(df_5, left_on='FL_DATE', right_on='Date', how='left')

# Add a column for weather delay cost
merged_df_4['Estimated_Weather_Delay_Cost'] = merged_df_4['WEATHER_DELAY'] * 101.18 + merged_df_4['NAS_DELAY'] * 101.18

# create a new dataframe that can be used later on
merged_df_5 = merged_df_4

# Drop the 'Date' column from the merged dataframe if you don't need it
merged_df_4 = merged_df_4.drop(columns=['Date'])

# Now, merged_df contains the Max Wind Gust and Average Wind Gust columns from df_2
merged_df_4

In [ ]:
# Drop unnecessary columns
merged_df_4.drop(columns=['Max Temperature', 'Min Temperature'], inplace=True)

In [ ]:
# Add delay cost data

In [ ]:
# Add a column for delay cost
#In 2022, the average cost of aircraft block (taxi plus airborne) time for U.S. passenger airlines was $101.18 per minute

merged_df_4['Estimated_Delay_Cost'] = merged_df_4['DEP_DELAY_NEW'] * 101.18

In [ ]:
# Optimize the dataframe for machine learning

In [ ]:
# Filter rows where the 'FL_DATE' is before 2022
merged_df_4 = merged_df_4[merged_df_4['FL_DATE'].dt.year < 2022]

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column
merged_df_4['DOY'] = pd.to_datetime(merged_df_4['FL_DATE']).dt.dayofyear
merged_df_4 = merged_df_4.drop(columns=['FL_DATE'])  # Remove the original 'FL_DATE' column

In [ ]:
columns_to_drop = ['CANCELLED', 'ORIGIN', 'DEP_DELAY_NEW', 'WEATHER_DELAY', 'NAS_DELAY', 'Delayed', 'Estimated_Delay_Cost', 'Delayed', 'Max Wind Gust','Max Precipitation']
merged_df_4 = merged_df_4.drop(columns_to_drop, axis=1)

In [ ]:
merged_df_4

In [ ]:
# Plots for inspection of variable relationships

In [ ]:
# plot percentage of weather delayed flights by carrier
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Set font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 20

# Calculate the percentage of Weather_Delayed being 0 and 1 for each OP_UNIQUE_CARRIER
carrier_percentage = merged_df_4.groupby(['OP_UNIQUE_CARRIER', 'Weather_Delayed']).size().unstack(fill_value=0)
carrier_percentage['Total'] = carrier_percentage[0] + carrier_percentage[1]
carrier_percentage['Percentage_0'] = (carrier_percentage[0] / carrier_percentage['Total']) * 100
carrier_percentage['Percentage_1'] = (carrier_percentage[1] / carrier_percentage['Total']) * 100

# Plot the percentage bar plots with total adding up to 100%
plt.figure(figsize=(12, 6))
sns.barplot(x='OP_UNIQUE_CARRIER', y='Percentage_0', data=carrier_percentage.reset_index(), color='#1f78b4', label='Weather_Delayed = 0')
sns.barplot(x='OP_UNIQUE_CARRIER', y='Percentage_1', data=carrier_percentage.reset_index(), color='#ff7f0e', bottom=carrier_percentage['Percentage_0'], label='Weather_Delayed = 1')
plt.xlabel('OP_UNIQUE_CARRIER [-]')
plt.ylabel('Percentage [%]')
plt.legend()
plt.show()


In [ ]:
# plot the relationship of airtime and flight status
import seaborn as sns
import matplotlib.pyplot as plt

# Set font to Times New Roman and set font size
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 20

# Create a seaborn boxplot with grey gridlines
sns.set(style="whitegrid", rc={"grid.linewidth": 0.5, "grid.color": "grey"})
ax = sns.boxplot(x='Weather_Delayed', y='AIR_TIME', data=merged_df_4, showfliers=False)

# Add vertical gridlines
ax.set(xticks=range(len(merged_df_4['Weather_Delayed'].unique())))
ax.xaxis.grid(True, linestyle='--', which='major', color='grey', alpha=0.7)

# Customize the plot
ax.set_xlabel('Weather_Delayed [-]')
ax.set_ylabel('AIR_TIME [Minutes]')

# Show the plot
plt.show()


In [ ]:
# Plot the relationships of weather variables and flight status
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams

# Set the font to Times New Roman
rcParams['font.family'] = 'times new roman'
rcParams['font.size'] = 12

# Filter the DataFrame to include only the relevant columns for boxplots
relevant_columns = [
    "Average Wind Gust",
    "Average Precipitation",
    "Max Temperature (°C)",
    "Min Temperature (°C)"
]

# Create subplots for boxplots without outliers
plt.figure(figsize=(12, 8))
for i, column in enumerate(relevant_columns):
    plt.subplot(3, 4, i+1)
    sns.boxplot(x=merged_df_4["Weather_Delayed"], y=merged_df_4[column], showfliers=False)

    # Set the x-axis and y-axis labels with Times New Roman font
    unit_label = "[°C]" if "Temperature" in column else "[m s-1]" if "Wind Gust" in column else "[kg m-2 s-1]"
    plt.xlabel("Weather Delayed [-]", fontdict={'family': 'times new roman', 'size': 12})
    plt.ylabel(f"{column.replace(' (°C)', '')} {unit_label}",
               fontdict={'family': 'times new roman', 'size': 12})

    # Add gray gridlines that are not dotted
    plt.grid(color='gray', linestyle='-', linewidth=0.25)

plt.tight_layout()
plt.show()


In [ ]:
# Plot the minimum and maximum values of the weather variables

In [ ]:
merged_df_4['Average Wind Gust'].min()

In [ ]:
merged_df_4['Average Wind Gust'].max()

In [ ]:
merged_df_4['Min Temperature (°C)'].min()

In [ ]:
merged_df_4['Min Temperature (°C)'].max()

In [ ]:
merged_df_4['Max Temperature (°C)'].min()

In [ ]:
merged_df_4['Max Temperature (°C)'].max()

In [ ]:
merged_df_4['Average Precipitation'].min()

In [ ]:
merged_df_4['Average Precipitation'].max()

In [ ]:
# Plot the balance of flights that are delayed vs on-time
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Set the font to Times New Roman
rcParams['font.family'] = 'times new roman'

# Filter the DataFrame to include only the relevant columns for histograms
relevant_columns = ["Weather_Delayed"]

# Create subplots for histograms
plt.figure(figsize=(12, 8), facecolor='white')  # Set the background color to white

for i, column in enumerate(relevant_columns):
    ax = plt.subplot(3, 4, i+1)
    
    # Extract the data as a numpy array
    data = merged_df_4[column].values
    
    # Separate the data based on outcomes
    delayed_data = data[data == 0]
    on_time_data = data[data == 1]
    
    # Set everything to dark blue
    delayed_color = 'darkblue'
    on_time_color = 'darkorange'
    
    # Plot separate histograms for each outcome
    plt.hist(delayed_data, bins=[-0.25, 0.25, 0.75, 1.25], color=delayed_color, alpha=0.7, label='Outcome 0')
    plt.hist(on_time_data, bins=[-0.25, 0.25, 0.75, 1.25], color=on_time_color, alpha=0.7, label='Outcome 1')
    
    # Set the frame (spines) to black and make them thinner
    for spine in ax.spines.values():
        spine.set_edgecolor('black')
        spine.set_linewidth(0.5)  # Adjust the line width for the frame
    
    # Set the x-axis and y-axis labels
    ax.set_xlabel("Weather_Delayed [-]", color='black', fontdict={'fontsize': 12})  # Set the x-axis label and color
    ax.set_ylabel("Frequency [-]", color='black', fontdict={'fontsize': 12})  # Set the y-axis label and color

    # Set x-axis ticks to only 0 and 1
    ax.set_xticks([0, 1])  
    ax.set_xticklabels([0, 1])  # Set the x-axis tick labels
    
    
    # Set gridlines to be in the background, grey, and not dotted
    ax.set_axisbelow(True)
    plt.grid(color='grey', linestyle='-', linewidth=0.5)
    
plt.tight_layout()

# Save the figure as a picture (e.g., histogram.png)
plt.savefig("histogram.png", facecolor='white')  # Set the background color when saving

# Display the figure (optional)
plt.show()


In [ ]:
(merged_df_4["Weather_Delayed"] == 0).sum()

In [ ]:
(merged_df_4["Weather_Delayed"] == 1).sum()

In [ ]:
# plot the correlation of variables
import seaborn as sns
import matplotlib.pyplot as plt

# Set the font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'

# Compute the correlation matrix
correlation_matrix = merged_df_4.corr()

# Create a heatmap with non-overlapping annotations and a title
plt.figure(figsize=(8, 6))  # Reduce the figure size

# Set the annot parameter to display values in each cell
# Use a diverging color map with red for positive values, blue for negative values, and center at 0 (white)
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm_r', center=0, annot_kws={"size": 10})

# Save the plot as a picture with higher resolution (e.g., correlation_matrix.png)
plt.savefig("correlation_matrix.png", dpi=50)  # Adjust the DPI as needed

plt.show()


In [ ]:
# Machine learning models

In [ ]:
# Data preprocessing

In [ ]:
#prepare data for modeling

# Select input features and target variable
selected_features = [
    'DOY', 'OP_UNIQUE_CARRIER', 'CRS_DEP_TIME', 'AIR_TIME', 'Day_Type',
    'Average Wind Gust', 'Average Precipitation', 'Min Temperature (°C)'
]
X = merged_df_4[selected_features]
y = merged_df_4['Weather_Delayed']

# Separate numerical and categorical columns
numerical_cols = ['CRS_DEP_TIME', 'AIR_TIME', 'Average Wind Gust', 'Average Precipitation', 'Min Temperature (°C)', 'DOY']
categorical_cols = ['OP_UNIQUE_CARRIER', 'Day_Type']

# Standardize numerical features
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Handle categorical features with one-hot encoding
encoder = OneHotEncoder(drop='first', sparse=False)
X_encoded = encoder.fit_transform(X[categorical_cols])
feature_names = encoder.get_feature_names_out(input_features=categorical_cols)
X_encoded = pd.DataFrame(X_encoded, columns=feature_names)
X = pd.concat([X.drop(columns=categorical_cols), X_encoded], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create a logistic regression model

In [ ]:
# Address class imbalance using RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy=0.6, random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)


# Create new interaction terms for the training set
for carrier in ['AA', 'AS', 'B6', 'DL', 'EV', 'HA', 'MQ', 'OH', 'OO', 'UA', 'US', 'VX', 'YX']:
    X_train_resampled[f'OP_CARRIER_DEP_TIME_Interact_{carrier}'] = X_train_resampled[f'OP_UNIQUE_CARRIER_{carrier}'] * X_train_resampled['CRS_DEP_TIME']
    X_train_resampled[f'OP_CARRIER_AIR_TIME_Interact_{carrier}'] = X_train_resampled[f'OP_UNIQUE_CARRIER_{carrier}'] * X_train_resampled['AIR_TIME']

# Create new interaction terms for the test set
for carrier in ['AA', 'AS', 'B6', 'DL', 'EV', 'HA', 'MQ', 'OH', 'OO', 'UA', 'US', 'VX', 'YX']:
    X_test[f'OP_CARRIER_DEP_TIME_Interact_{carrier}'] = X_test[f'OP_UNIQUE_CARRIER_{carrier}'] * X_test['CRS_DEP_TIME']
    X_test[f'OP_CARRIER_AIR_TIME_Interact_{carrier}'] = X_test[f'OP_UNIQUE_CARRIER_{carrier}'] * X_test['AIR_TIME']


# Model 

# Define fixed hyperparameters for the logistic regression model
logistic_params = {
     'C': 1000,  # Regularization parameter
    'max_iter': 500  # Maximum number of iterations
}

# Initialize the logistic regression model with fixed hyperparameters
model = LogisticRegression(**logistic_params)

# Define StratifiedKFold with 3 folds
stratified_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Initialize lists to store evaluation results for each fold
accuracies = []
class_reports = []
confusion_matrices = []

# Perform cross-validation
for train_idx, test_idx in stratified_cv.split(X_train_resampled, y_train_resampled):
    X_train_fold, X_val_fold = X_train_resampled.iloc[train_idx], X_train_resampled.iloc[test_idx]
    y_train_fold, y_val_fold = y_train_resampled.iloc[train_idx], y_train_resampled.iloc[test_idx]

    # Train the model on the training fold
    model.fit(X_train_fold, y_train_fold)

    # Make predictions on the validation fold
    y_val_pred = model.predict(X_val_fold)

    # Evaluate the model on the validation fold
    accuracy_fold = accuracy_score(y_val_fold, y_val_pred)
    class_report_fold = classification_report(y_val_fold, y_val_pred)
    confusion_mat_fold = confusion_matrix(y_val_fold, y_val_pred)

    # Store the evaluation results for this fold
    accuracies.append(accuracy_fold)
    class_reports.append(class_report_fold)
    confusion_matrices.append(confusion_mat_fold)

# Calculate the average accuracy and report for all folds
average_accuracy = sum(accuracies) / len(accuracies)
average_class_report = "\n\n".join(class_reports)
average_confusion_matrix = sum(confusion_matrices)

print(f'Average Accuracy: {average_accuracy}')
print('Average Classification Report:\n', average_class_report)
print('Average Confusion Matrix:\n', average_confusion_matrix)

# Now you can evaluate the model on the test set if needed
y_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
test_class_report = classification_report(y_test, y_pred)
test_confusion_mat = confusion_matrix(y_test, y_pred)

print(f'Test Accuracy: {test_accuracy}')
print('Test Classification Report:\n', test_class_report)
print('Test Confusion Matrix:\n', test_confusion_mat)



In [ ]:
# Create a random forest model

In [ ]:
# Initialize stratified k-fold cross-validator
stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)  # Adjust the number of splits as needed

# Initialize lists to store evaluation metrics across all folds
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
roc_auc_list = []
conf_matrix_list = []

# Model Building and Cross-validation
for train_index, test_index in stratified_kfold.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Address Class Imbalance with SMOTE (Synthetic Minority Over-sampling Technique)
    smote = SMOTE(sampling_strategy=0.6, random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

    # Create interaction terms
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    X_resampled_poly = poly.fit_transform(X_resampled)

    # Initialize and train a random forest classifier
    clf = RandomForestClassifier(n_estimators=150, max_depth=30, random_state=42)
    clf.fit(X_resampled_poly, y_resampled)

    # Create interaction terms for test data as well
    X_test_poly = poly.transform(X_test)

    # Make predictions
    y_pred = clf.predict(X_test_poly)

    # Evaluate the model using different metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test_poly)[:, 1])
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Append metrics to the respective lists
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    roc_auc_list.append(roc_auc)
    conf_matrix_list.append(conf_matrix)

# Calculate the average metrics across all folds
avg_accuracy = sum(accuracy_list) / len(accuracy_list)
avg_precision = sum(precision_list) / len(precision_list)
avg_recall = sum(recall_list) / len(recall_list)
avg_f1 = sum(f1_list) / len(f1_list)
avg_roc_auc = sum(roc_auc_list) / len(roc_auc_list)

# Calculate the average confusion matrix
avg_conf_matrix = sum(conf_matrix_list) / len(conf_matrix_list)

# Print the average metrics
print(f"Average Accuracy: {avg_accuracy:.2f}")
print(f"Average Precision: {avg_precision:.2f}")
print(f"Average Recall: {avg_recall:.2f}")
print(f"Average F1 Score: {avg_f1:.2f}")
print(f"Average ROC AUC: {avg_roc_auc:.2f}")

# Print the average confusion matrix
print("Average Confusion Matrix:")
print(avg_conf_matrix)


In [ ]:
# Gradient Boosting

In [ ]:
# run

# Address class imbalance using SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)

# Create a StratifiedKFold object for stratified cross-validation
stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Define the Gradient Boosting Classifier with your chosen hyperparameters
clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.4, max_depth=5, random_state=42)

# Initialize variables to store results
accuracies = []
roc_auc_scores = []  # Added for ROC AUC
reports = []
confusion_matrices = []  # Added for confusion matrices

# Perform stratified cross-validation
for train_index, test_index in stratified_kfold.split(X_resampled, y_resampled):
    X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
    y_train, y_test = y_resampled.iloc[train_index], y_resampled.iloc[test_index]

    # Train the model on the training data
    clf.fit(X_train, y_train)

    # Make predictions on the test data and evaluate the model
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])  # Calculate ROC AUC
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)  # Calculate confusion matrix

    accuracies.append(accuracy)
    roc_auc_scores.append(roc_auc)  # Store ROC AUC score
    reports.append(report)
    confusion_matrices.append(cm)  # Store confusion matrix

# Calculate the mean accuracy and ROC AUC, and print the classification reports
mean_accuracy = sum(accuracies) / len(accuracies)
mean_roc_auc = sum(roc_auc_scores) / len(roc_auc_scores)
print("Mean Accuracy:", mean_accuracy)
print("Mean ROC AUC:", mean_roc_auc)

for i, report in enumerate(reports):
    print(f"Classification Report for Fold {i + 1}:\n", report)

# Print the confusion matrices for each fold
for i, cm in enumerate(confusion_matrices):
    print(f"Confusion Matrix for Fold {i + 1}:\n", cm)


In [ ]:
# Neural network for classification and regression

In [ ]:
# Select features and target variables
regression_features = [
    'DOY', 'OP_UNIQUE_CARRIER', 'CRS_DEP_TIME', 'AIR_TIME', 'Day_Type',
    'Average Wind Gust', 'Average Precipitation', 'Min Temperature (°C)',
    'Weather_Delayed'
]

regression_target = 'Estimated_Weather_Delay_Cost'

# Separate features and targets
X_reg = merged_df_4[regression_features]
y_reg = merged_df_4[regression_target]

# Separate numerical and categorical columns for regression
numerical_cols_reg = ['CRS_DEP_TIME', 'AIR_TIME', 'Average Wind Gust', 'Average Precipitation', 'Min Temperature (°C)', 'DOY']
categorical_cols_reg = ['OP_UNIQUE_CARRIER', 'Day_Type']

# Standardize numerical features for regression
scaler_reg = StandardScaler()
X_reg[numerical_cols_reg] = scaler_reg.fit_transform(X_reg[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
encoder_reg = OneHotEncoder(drop='first', sparse=False)
X_encoded_reg = encoder_reg.fit_transform(X_reg[categorical_cols_reg])
feature_names_reg = encoder_reg.get_feature_names_out(input_features=categorical_cols_reg)
X_encoded_reg = pd.DataFrame(X_encoded_reg, columns=feature_names_reg)
X_reg = pd.concat([X_reg.drop(columns=categorical_cols_reg), X_encoded_reg], axis=1)

# Define a custom Huber activation function
def huber_activation(x):
    return tf.where(tf.abs(x) < 1, x**2 / 2, tf.abs(x) - 0.5)

# Neural network architecture for regression with Huber activation and loss
input_reg = Input(shape=(X_reg.shape[1],))
hidden_layer_reg = Dense(64, activation=huber_activation)(input_reg)
output_reg = Dense(1, activation='linear', name='output_reg')(hidden_layer_reg)

# Add an additional hidden layer with more neurons
hidden_layer_reg_2 = Dense(128, activation=huber_activation)(hidden_layer_reg)
output_reg = Dense(1, activation='linear', name='output_reg')(hidden_layer_reg_2)

# Compile the model with Huber loss
model = Model(inputs=input_reg, outputs=output_reg)
model.compile(optimizer='adam', loss=Huber(), metrics=['mae', 'mse'])

# Train the model
X_train, X_test, y_train, y_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on test data
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R^2): {r2}')


In [ ]:
# Regression to forecast Estimated_Weather_Delay_Cost

In [ ]:
# Select features and target variable for regression
regression_features = [
    'DOY', 'OP_UNIQUE_CARRIER', 'CRS_DEP_TIME', 'AIR_TIME', 'Day_Type',
    'Average Wind Gust', 'Average Precipitation', 'Min Temperature (°C)',
    'Weather_Delayed'  # Include Weather_Delayed as a feature for regression
]
regression_target = 'Estimated_Weather_Delay_Cost'

# Separate numerical and categorical columns for regression
regression_numerical_cols = ['CRS_DEP_TIME', 'AIR_TIME', 'Average Wind Gust', 'Average Precipitation', 'Min Temperature (°C)', 'DOY' 
                             ,'Weather_Delayed'
                            ]
regression_categorical_cols = ['OP_UNIQUE_CARRIER', 'Day_Type']

# Select features and target variable for regression
X_reg = merged_df_4[regression_features]
y_reg = merged_df_4[regression_target]

# Separate numerical and categorical columns for regression
numerical_cols_reg = ['CRS_DEP_TIME', 'AIR_TIME', 'Average Wind Gust', 'Average Precipitation', 'Min Temperature (°C)', 'DOY', 
                      'Weather_Delayed'
                     ]
categorical_cols_reg = ['OP_UNIQUE_CARRIER', 'Day_Type']

# Standardize numerical features for regression
scaler_reg = StandardScaler()
X_reg[numerical_cols_reg] = scaler_reg.fit_transform(X_reg[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
encoder_reg = OneHotEncoder(drop='first', sparse=False)
X_encoded_reg = encoder_reg.fit_transform(X_reg[categorical_cols_reg])
feature_names_reg = encoder_reg.get_feature_names_out(input_features=categorical_cols_reg)
X_encoded_reg = pd.DataFrame(X_encoded_reg, columns=feature_names_reg)
X_reg = pd.concat([X_reg.drop(columns=categorical_cols_reg), X_encoded_reg], axis=1)


# Create a KFold object for cross-validation for regression
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Define the Gradient Boosting Regressor with quantile loss (Huber approximation)
regressor = GradientBoostingRegressor(n_estimators=10, learning_rate=0.4, max_depth=12, loss='quantile', alpha=0.5, random_state=42)

# Initialize variables to store results
mae_scores = []
mse_scores = []
r2_scores = []

# Perform cross-validation for regression
for train_index, test_index in kf.split(X_reg):
    X_train_reg, X_test_reg = X_reg.iloc[train_index], X_reg.iloc[test_index]
    y_train_reg, y_test_reg = y_reg.iloc[train_index], y_reg.iloc[test_index]

    # Train the model on the training data
    regressor.fit(X_train_reg, y_train_reg)

    # Make predictions on the test data and evaluate the model
    y_pred_reg = regressor.predict(X_test_reg)
    
    # Calculate regression metrics
    mae = mean_absolute_error(y_test_reg, y_pred_reg)
    mse = mean_squared_error(y_test_reg, y_pred_reg)
    r2 = r2_score(y_test_reg, y_pred_reg)

    # Store regression metrics
    mae_scores.append(mae)
    mse_scores.append(mse)
    r2_scores.append(r2)

# Calculate the mean regression metrics and print the results
mean_mae = sum(mae_scores) / len(mae_scores)
mean_mse = sum(mse_scores) / len(mse_scores)
mean_r2 = sum(r2_scores) / len(r2_scores)
print("Mean MAE:", mean_mae)
print("Mean MSE:", mean_mse)
print("Mean R^2:", mean_r2)


In [ ]:
# Bias correction of CMIP6 data / Quantile delta mapping

In [ ]:
# Quantile mapping and quantile delta mapping are two techniques used in climate science and hydrology 
# to adjust simulated or modeled data to better match observed data. 
# These methods are often employed to correct biases or discrepancies between model outputs and real-world observations

In [ ]:
# Loading ERA5 datasets

In [ ]:
# Min temperature 2003-2014

In [ ]:
# inspect data
mintemp_ds_314 = nc.Dataset('./ERA5_data/ERA5_min2mtemp_3-14.nc', 'r')
mintemp_ds_314

In [ ]:
# List dimensions
print("Dimensions:")
for dimname, dimobj in mintemp_ds_314.dimensions.items():
    print(f"{dimname}: {len(dimobj)}")

# List variables
print("\nVariables:")
for varname, varobj in mintemp_ds_314.variables.items():
    print(f"{varname}: {varobj.shape} - {varobj.dtype}")

# List global attributes
print("\nGlobal Attributes:")
for attrname in mintemp_ds_314.ncattrs():
    print(f"{attrname}: {getattr(mintemp_ds_314, attrname)}")

In [ ]:
# Load the netCDF dataset
hist_mintemp_ds = nc.Dataset('./ERA5_data/ERA5_min2mtemp_3-14.nc', 'r')

# Extract time and values
time_values = hist_mintemp_ds.variables['time'][:]
hist_mintemp_values = hist_mintemp_ds.variables['mn2t'][:, 0, 0]

# Filter data until August 31, 2023
end_date = pd.to_datetime("2014-12-31")
filtered_indices = time_values <= pd.Timestamp(end_date).timestamp()

# Convert time values to a human-readable format (you can adjust this as needed)
hours = time_values[filtered_indices] % 24
dates = pd.to_datetime(time_values[filtered_indices], unit='h', origin='1900-01-01')
dates += pd.to_timedelta(hours, unit='h')

# Create a DataFrame
hist_mintemp_df = pd.DataFrame({'Date': dates, 'Hist Min Temperature': hist_mintemp_values[filtered_indices]})

# Group by date and calculate the min for each day
hist_mintemp_df = hist_mintemp_df.groupby(hist_mintemp_df['Date'].dt.date)['Hist Min Temperature'].agg(['min']).reset_index()
hist_mintemp_df.columns = ['Date', 'Hist Min Temperature']

# Print the DataFrame with min values per day
print(hist_mintemp_df)

In [ ]:
# Convert temeprature to °C
hist_mintemp_df['Hist Min Temperature (°C)'] = hist_mintemp_df['Hist Min Temperature'] - 273.15

In [ ]:
# Convert the 'Date' column to a datetime type (if it's not already)
hist_mintemp_df['Date'] = pd.to_datetime(hist_mintemp_df['Date'])

# Filter the DataFrame to include only rows until 2014-12-31
hist_mintemp_df = hist_mintemp_df[hist_mintemp_df['Date'] <= '2014-12-31']
hist_mintemp_df

In [ ]:
# Windspeed 2003-2014

In [ ]:
# inspect data
hist_wind_ds = nc.Dataset('./ERA5_data/ERA5_10mwindgust_3-14.nc', 'r')
hist_wind_ds

In [ ]:
# List dimensions
print("Dimensions:")
for dimname, dimobj in hist_wind_ds.dimensions.items():
    print(f"{dimname}: {len(dimobj)}")

# List variables
print("\nVariables:")
for varname, varobj in hist_wind_ds.variables.items():
    print(f"{varname}: {varobj.shape} - {varobj.dtype}")

# List global attributes
print("\nGlobal Attributes:")
for attrname in hist_wind_ds.ncattrs():
    print(f"{attrname}: {getattr(hist_wind_ds, attrname)}")


In [ ]:
# Load the netCDF dataset
hist_wind_ds = nc.Dataset('./ERA5_data/ERA5_10mwindgust_3-14.nc', 'r')

# Extract time and wind gust values
time_values = hist_wind_ds.variables['time'][:]
hist_wind_gust_values = hist_wind_ds.variables['fg10'][:, 0, 0]

# Filter data until August 31, 2023
end_date = pd.to_datetime("2014-12-31")
filtered_indices = time_values <= pd.Timestamp(end_date).timestamp()

# Convert time values to a human-readable format (you can adjust this as needed)
hours = time_values[filtered_indices] % 24
dates = pd.to_datetime(time_values[filtered_indices], unit='h', origin='1900-01-01')
dates += pd.to_timedelta(hours, unit='h')

# Create a DataFrame
hist_wind_df = pd.DataFrame({'Date': dates, 'Hist Wind Gust': hist_wind_gust_values[filtered_indices]})

# Group by date and calculate both the maximum and average wind gust values for each day
hist_wind_df = hist_wind_df.groupby(hist_wind_df['Date'].dt.date)['Hist Wind Gust'].agg(['mean']).reset_index()
hist_wind_df.columns = ['Date', 'Hist Average Wind Gust']

# Print the DataFrame with both maximum and average wind gust values per day
print(hist_wind_df)

In [ ]:
# Convert the 'Date' column to a datetime type (if it's not already)
hist_wind_df['Date'] = pd.to_datetime(hist_wind_df['Date'])

# Filter the DataFrame to include only rows until 2014-12-31
hist_wind_df = hist_wind_df[hist_wind_df['Date'] <= '2014-12-31']
hist_wind_df

In [ ]:
# Historical precipitation rate 2003-2014

In [ ]:
# inspect data
hist_prec_ds = nc.Dataset('./ERA5_data/ERA5_maxtotalprec_3-14.nc', 'r')
hist_prec_ds

In [ ]:
# List dimensions
print("Dimensions:")
for dimname, dimobj in hist_prec_ds.dimensions.items():
    print(f"{dimname}: {len(dimobj)}")

# List variables
print("\nVariables:")
for varname, varobj in hist_prec_ds.variables.items():
    print(f"{varname}: {varobj.shape} - {varobj.dtype}")

# List global attributes
print("\nGlobal Attributes:")
for attrname in hist_prec_ds.ncattrs():
    print(f"{attrname}: {getattr(hist_prec_ds, attrname)}")

In [ ]:
# Load the netCDF dataset
hist_prec_ds = nc.Dataset('./ERA5_data/ERA5_maxtotalprec_3-14.nc', 'r')

# Extract time and wind gust values
time_values = hist_prec_ds.variables['time'][:]
hist_prec_values = hist_prec_ds.variables['mxtpr'][:, 0, 0]

# Filter data until August 31, 2023
end_date = pd.to_datetime("2014-12-31")
filtered_indices = time_values <= pd.Timestamp(end_date).timestamp()

# Convert time values to a human-readable format (you can adjust this as needed)
hours = time_values[filtered_indices] % 24
dates = pd.to_datetime(time_values[filtered_indices], unit='h', origin='1900-01-01')
dates += pd.to_timedelta(hours, unit='h')

# Create a DataFrame
hist_prec_df = pd.DataFrame({'Date': dates, 'Hist Precipitation': hist_prec_values[filtered_indices]})

# Group by date and calculate both the maximum and average wind gust values for each day
hist_prec_df = hist_prec_df.groupby(hist_prec_df['Date'].dt.date)['Hist Precipitation'].agg(['mean']).reset_index()
hist_prec_df.columns = ['Date', 'Hist Average Precipitation']

# Print the DataFrame with both maximum and average wind gust values per day
print(hist_prec_df)


In [ ]:
# Convert the 'Date' column to a datetime type (if it's not already)
hist_prec_df['Date'] = pd.to_datetime(hist_prec_df['Date'])

# Filter the DataFrame to include only rows until 2014-12-31
hist_prec_df = hist_prec_df[hist_prec_df['Date'] <= '2014-12-31']
hist_prec_df

In [ ]:
# Load CMIP6 data 2003-2014

In [ ]:
# Min temp

In [ ]:
# Open the NetCDF file for reading
with nc.Dataset('./CMIP6_data/hist_mintemp_3-14.nc', 'r') as data:
    # Get a list of variable names
    variable_names = list(data.variables.keys())

# Print the variable names
print("Variables in the NetCDF file:")
for var_name in variable_names:
    print(var_name)


In [ ]:
# inspect dataset
CMIP_hist_mintemp_ds = nc.Dataset('./CMIP6_data/hist_mintemp_3-14.nc', 'r')
CMIP_hist_mintemp_ds

In [ ]:
# Define the latitude and longitude coordinates for New York JFK Airport
ny_jfk_lat = 40.6413
ny_jfk_lon = -73.7781

# Find the nearest latitude and longitude indices in the dataset
lat_values = CMIP_hist_mintemp_ds.variables['lat'][:]
lon_values = CMIP_hist_mintemp_ds.variables['lon'][:]
lat_index = abs(lat_values - ny_jfk_lat).argmin()
lon_index = abs(lon_values - ny_jfk_lon).argmin()

# Extract time and temperature data
time_data = CMIP_hist_mintemp_ds.variables['time'][:]
temperature_data = CMIP_hist_mintemp_ds.variables['tasmin'][:, lat_index, lon_index]

# Convert the time data to a more understandable format
time_units = CMIP_hist_mintemp_ds.variables['time'].units
time_calendar = CMIP_hist_mintemp_ds.variables['time'].calendar
time_in_days = nc.num2date(time_data, units=time_units, calendar=time_calendar)

# Create a Pandas DataFrame
CMIP_hist_mintemp_df = pd.DataFrame({'Day': time_in_days, 'CMIP Hist Temperature': temperature_data})

CMIP_hist_mintemp_df['Day'] = pd.to_datetime(CMIP_hist_mintemp_df['Day'].astype(str)).dt.date

# Display the first few rows of the DataFrame
CMIP_hist_mintemp_df


In [ ]:
# change format to datetime
CMIP_hist_mintemp_df['Day'] = pd.to_datetime(CMIP_hist_mintemp_df['Day'].astype(str)).dt.date

In [ ]:
# convert temperature to °C
CMIP_hist_mintemp_df['CMIP Hist Temperature (C°)'] = CMIP_hist_mintemp_df['CMIP Hist Temperature'] - 273.15

In [ ]:
CMIP_hist_mintemp_df

In [ ]:
# wind gust

In [ ]:
# Open the NetCDF file for reading
with nc.Dataset('./CMIP6_data/hist_windsp_3-14.nc', 'r') as data:
    # Get a list of variable names
    variable_names = list(data.variables.keys())

# Print the variable names
print("Variables in the NetCDF file:")
for var_name in variable_names:
    print(var_name)

In [ ]:
# inspect dataset
CMIP_hist_wind_ds = nc.Dataset('./CMIP6_data/hist_windsp_3-14.nc', 'r')
CMIP_hist_wind_ds

In [ ]:
# Define the latitude and longitude coordinates for New York JFK Airport
ny_jfk_lat = 40.6413
ny_jfk_lon = -73.7781

# Find the nearest latitude and longitude indices in the dataset
lat_values = CMIP_hist_wind_ds.variables['lat'][:]
lon_values = CMIP_hist_wind_ds.variables['lon'][:]
lat_index = abs(lat_values - ny_jfk_lat).argmin()
lon_index = abs(lon_values - ny_jfk_lon).argmin()

# Extract time and temperature data
time_data = CMIP_hist_wind_ds.variables['time'][:]
wind_data = CMIP_hist_wind_ds.variables['sfcWind'][:, lat_index, lon_index]

# Convert the time data to a more understandable format
time_units = CMIP_hist_wind_ds.variables['time'].units
time_calendar = CMIP_hist_wind_ds.variables['time'].calendar
time_in_days = nc.num2date(time_data, units=time_units, calendar=time_calendar)

# Create a Pandas DataFrame
CMIP_hist_wind_df = pd.DataFrame({'Day': time_in_days, 'CMIP Hist Wind': wind_data})

# Display the first few rows of the DataFrame
CMIP_hist_wind_df


In [ ]:
# convert fromat to datetime
CMIP_hist_wind_df['Day'] = pd.to_datetime(CMIP_hist_wind_df['Day'].astype(str)).dt.date

In [ ]:
CMIP_hist_wind_df

In [ ]:
# Precipitation

In [ ]:
# Open the NetCDF file for reading
with nc.Dataset('./CMIP6_data/hist_prec_3-14.nc', 'r') as data:
    # Get a list of variable names
    variable_names = list(data.variables.keys())

# Print the variable names
print("Variables in the NetCDF file:")
for var_name in variable_names:
    print(var_name)

In [ ]:
# inspect dataset
CMIP_hist_prec_ds = nc.Dataset('./CMIP6_data/hist_prec_3-14.nc', 'r')
CMIP_hist_prec_ds

In [ ]:
# Define the latitude and longitude coordinates for New York JFK Airport
ny_jfk_lat = 40.6413
ny_jfk_lon = -73.7781

# Find the nearest latitude and longitude indices in the dataset
lat_values = CMIP_hist_prec_ds.variables['lat'][:]
lon_values = CMIP_hist_prec_ds.variables['lon'][:]
lat_index = abs(lat_values - ny_jfk_lat).argmin()
lon_index = abs(lon_values - ny_jfk_lon).argmin()

# Extract time and temperature data
time_data = CMIP_hist_prec_ds.variables['time'][:]
prec_data = CMIP_hist_prec_ds.variables['pr'][:, lat_index, lon_index]

# Convert the time data to a more understandable format
time_units = CMIP_hist_prec_ds.variables['time'].units
time_calendar = CMIP_hist_prec_ds.variables['time'].calendar
time_in_days = nc.num2date(time_data, units=time_units, calendar=time_calendar)

# Create a Pandas DataFrame
CMIP_hist_prec_df = pd.DataFrame({'Day': time_in_days, 'CMIP Hist Precipitation': prec_data})

CMIP_hist_prec_df['Day'] = pd.to_datetime(CMIP_hist_prec_df['Day'].astype(str)).dt.date

# Display the first few rows of the DataFrame
CMIP_hist_prec_df


In [ ]:
# Load the Future data SSP 1.26

In [ ]:
# Min temp

In [ ]:
# Open the NetCDF file for reading
with nc.Dataset('./CMIP6_data/ssp126_mintemp_15-30.nc', 'r') as data:
    # Get a list of variable names
    variable_names = list(data.variables.keys())

# Print the variable names
print("Variables in the NetCDF file:")
for var_name in variable_names:
    print(var_name)


In [ ]:
# inspect dataset
CMIP_SSP126_mintemp_ds = nc.Dataset('./CMIP6_data/ssp126_mintemp_15-30.nc', 'r')
CMIP_SSP126_mintemp_ds

In [ ]:
# Define the latitude and longitude coordinates for New York JFK Airport
ny_jfk_lat = 40.6413
ny_jfk_lon = -73.7781

# Find the nearest latitude and longitude indices in the dataset
lat_values = CMIP_SSP126_mintemp_ds.variables['lat'][:]
lon_values = CMIP_SSP126_mintemp_ds.variables['lon'][:]
lat_index = abs(lat_values - ny_jfk_lat).argmin()
lon_index = abs(lon_values - ny_jfk_lon).argmin()

# Extract time and temperature data
time_data = CMIP_SSP126_mintemp_ds.variables['time'][:]
temperature_data = CMIP_SSP126_mintemp_ds.variables['tasmin'][:, lat_index, lon_index]

# Convert the time data to a more understandable format
time_units = CMIP_SSP126_mintemp_ds.variables['time'].units
time_calendar = CMIP_SSP126_mintemp_ds.variables['time'].calendar
time_in_days = nc.num2date(time_data, units=time_units, calendar=time_calendar)

# Create a Pandas DataFrame
CMIP_SSP126_mintemp_df = pd.DataFrame({'Day': time_in_days, 'CMIP SSP126 Temperature': temperature_data})

CMIP_SSP126_mintemp_df['Day'] = pd.to_datetime(CMIP_SSP126_mintemp_df['Day'].astype(str)).dt.date

CMIP_SSP126_mintemp_df['Day'] = pd.to_datetime(CMIP_SSP126_mintemp_df['Day'])

CMIP_SSP126_mintemp_df = CMIP_SSP126_mintemp_df[CMIP_SSP126_mintemp_df['Day'] >= '2019-01-01']

CMIP_SSP126_mintemp_df['CMIP SSP126 Temperature (°C)'] = CMIP_SSP126_mintemp_df['CMIP SSP126 Temperature'] - 273.15

CMIP_SSP126_mintemp_df.drop('CMIP SSP126 Temperature', axis=1, inplace=True)

# Display the first few rows of the DataFrame
CMIP_SSP126_mintemp_df


In [ ]:
# Wind gust

In [ ]:
# Open the NetCDF file for reading
with nc.Dataset('./CMIP6_data/ssp126_windsp_15-30.nc', 'r') as data:
    # Get a list of variable names
    variable_names = list(data.variables.keys())

# Print the variable names
print("Variables in the NetCDF file:")
for var_name in variable_names:
    print(var_name)


In [ ]:
# inspect dataset
CMIP_SSP126_wg_ds = nc.Dataset('./CMIP6_data/ssp126_windsp_15-30.nc', 'r')
CMIP_SSP126_wg_ds

In [ ]:
# Define the latitude and longitude coordinates for New York JFK Airport
ny_jfk_lat = 40.6413
ny_jfk_lon = -73.7781

# Find the nearest latitude and longitude indices in the dataset
lat_values = CMIP_SSP126_wg_ds.variables['lat'][:]
lon_values = CMIP_SSP126_wg_ds.variables['lon'][:]
lat_index = abs(lat_values - ny_jfk_lat).argmin()
lon_index = abs(lon_values - ny_jfk_lon).argmin()

# Extract time and temperature data
time_data = CMIP_SSP126_wg_ds.variables['time'][:]
wg_data = CMIP_SSP126_wg_ds.variables['sfcWind'][:, lat_index, lon_index]

# Convert the time data to a more understandable format
time_units = CMIP_SSP126_wg_ds.variables['time'].units
time_calendar = CMIP_SSP126_wg_ds.variables['time'].calendar
time_in_days = nc.num2date(time_data, units=time_units, calendar=time_calendar)

# Create a Pandas DataFrame
CMIP_SSP126_wg_df = pd.DataFrame({'Day': time_in_days, 'CMIP SSP126 Wind Gust': wg_data})

CMIP_SSP126_wg_df['Day'] = pd.to_datetime(CMIP_SSP126_wg_df['Day'].astype(str)).dt.date

CMIP_SSP126_wg_df['Day'] = pd.to_datetime(CMIP_SSP126_wg_df['Day'])

CMIP_SSP126_wg_df = CMIP_SSP126_wg_df[CMIP_SSP126_wg_df['Day'] >= '2019-01-01']

# Display the first few rows of the DataFrame
CMIP_SSP126_wg_df

In [ ]:
# Future precipitation

In [ ]:
# Open the NetCDF file for reading
with nc.Dataset('./CMIP6_data/ssp126_prec_15-30.nc', 'r') as data:
    # Get a list of variable names
    variable_names = list(data.variables.keys())

# Print the variable names
print("Variables in the NetCDF file:")
for var_name in variable_names:
    print(var_name)


In [ ]:
# inspect dataset
CMIP_SSP126_prec_ds = nc.Dataset('./CMIP6_data/ssp126_prec_15-30.nc', 'r')
CMIP_SSP126_prec_ds

In [ ]:
# Define the latitude and longitude coordinates for New York JFK Airport
ny_jfk_lat = 40.6413
ny_jfk_lon = -73.7781

# Find the nearest latitude and longitude indices in the dataset
lat_values = CMIP_SSP126_prec_ds.variables['lat'][:]
lon_values = CMIP_SSP126_prec_ds.variables['lon'][:]
lat_index = abs(lat_values - ny_jfk_lat).argmin()
lon_index = abs(lon_values - ny_jfk_lon).argmin()

# Extract time and temperature data
time_data = CMIP_SSP126_prec_ds.variables['time'][:]
prec_data = CMIP_SSP126_prec_ds.variables['pr'][:, lat_index, lon_index]

# Convert the time data to a more understandable format
time_units = CMIP_SSP126_prec_ds.variables['time'].units
time_calendar = CMIP_SSP126_prec_ds.variables['time'].calendar
time_in_days = nc.num2date(time_data, units=time_units, calendar=time_calendar)

# Create a Pandas DataFrame
CMIP_SSP126_prec_df = pd.DataFrame({'Day': time_in_days, 'CMIP SSP126 Precipitation Rate': prec_data})

CMIP_SSP126_prec_df['Day'] = pd.to_datetime(CMIP_SSP126_prec_df['Day'].astype(str)).dt.date

CMIP_SSP126_prec_df['Day'] = pd.to_datetime(CMIP_SSP126_prec_df['Day'])

CMIP_SSP126_prec_df = CMIP_SSP126_prec_df[CMIP_SSP126_prec_df['Day'] >= '2019-01-01']

# Display the first few rows of the DataFrame
CMIP_SSP126_prec_df


In [ ]:
# SSP 585 min temp

In [ ]:
# Open the NetCDF file for reading
with nc.Dataset('./CMIP6_data/ssp585_mintemp_15-30.nc', 'r') as data:
    # Get a list of variable names
    variable_names = list(data.variables.keys())

# Print the variable names
print("Variables in the NetCDF file:")
for var_name in variable_names:
    print(var_name)


In [ ]:
# inspect dataset
CMIP_SSP585_mintemp_ds = nc.Dataset('./CMIP6_data/ssp585_mintemp_15-30.nc', 'r')
CMIP_SSP585_mintemp_ds

In [ ]:
# Define the latitude and longitude coordinates for New York JFK Airport
ny_jfk_lat = 40.6413
ny_jfk_lon = -73.7781

# Find the nearest latitude and longitude indices in the dataset
lat_values = CMIP_SSP585_mintemp_ds.variables['lat'][:]
lon_values = CMIP_SSP585_mintemp_ds.variables['lon'][:]
lat_index = abs(lat_values - ny_jfk_lat).argmin()
lon_index = abs(lon_values - ny_jfk_lon).argmin()

# Extract time and temperature data
time_data = CMIP_SSP585_mintemp_ds.variables['time'][:]
temperature_data = CMIP_SSP585_mintemp_ds.variables['tasmin'][:, lat_index, lon_index]

# Convert the time data to a more understandable format
time_units = CMIP_SSP585_mintemp_ds.variables['time'].units
time_calendar = CMIP_SSP585_mintemp_ds.variables['time'].calendar
time_in_days = nc.num2date(time_data, units=time_units, calendar=time_calendar)

# Create a Pandas DataFrame
CMIP_SSP585_mintemp_df = pd.DataFrame({'Day': time_in_days, 'CMIP SSP585 Temperature': temperature_data})

CMIP_SSP585_mintemp_df['Day'] = pd.to_datetime(CMIP_SSP585_mintemp_df['Day'].astype(str)).dt.date

CMIP_SSP585_mintemp_df['Day'] = pd.to_datetime(CMIP_SSP585_mintemp_df['Day'])

CMIP_SSP585_mintemp_df = CMIP_SSP585_mintemp_df[CMIP_SSP585_mintemp_df['Day'] >= '2019-01-01']

CMIP_SSP585_mintemp_df['CMIP SSP585 Temperature (°C)'] = CMIP_SSP585_mintemp_df['CMIP SSP585 Temperature'] - 273.15

CMIP_SSP585_mintemp_df.drop('CMIP SSP585 Temperature', axis=1, inplace=True)

# Display the first few rows of the DataFrame
CMIP_SSP585_mintemp_df


In [ ]:
# SSP585 - wind gust

In [ ]:
# Open the NetCDF file for reading
with nc.Dataset('./CMIP6_data/ssp585_windsp_15-30.nc', 'r') as data:
    # Get a list of variable names
    variable_names = list(data.variables.keys())

# Print the variable names
print("Variables in the NetCDF file:")
for var_name in variable_names:
    print(var_name)


In [ ]:
# inspect dataset
CMIP_SSP585_wg_ds = nc.Dataset('./CMIP6_data/ssp585_windsp_15-30.nc', 'r')
CMIP_SSP585_wg_ds

In [ ]:
# Define the latitude and longitude coordinates for New York JFK Airport
ny_jfk_lat = 40.6413
ny_jfk_lon = -73.7781

# Find the nearest latitude and longitude indices in the dataset
lat_values = CMIP_SSP585_wg_ds.variables['lat'][:]
lon_values = CMIP_SSP585_wg_ds.variables['lon'][:]
lat_index = abs(lat_values - ny_jfk_lat).argmin()
lon_index = abs(lon_values - ny_jfk_lon).argmin()

# Extract time and temperature data
time_data = CMIP_SSP585_wg_ds.variables['time'][:]
wg_data = CMIP_SSP585_wg_ds.variables['sfcWind'][:, lat_index, lon_index]

# Convert the time data to a more understandable format
time_units = CMIP_SSP585_wg_ds.variables['time'].units
time_calendar = CMIP_SSP585_wg_ds.variables['time'].calendar
time_in_days = nc.num2date(time_data, units=time_units, calendar=time_calendar)

# Create a Pandas DataFrame
CMIP_SSP585_wg_df = pd.DataFrame({'Day': time_in_days, 'CMIP SSP585 Wind Gust': wg_data})

CMIP_SSP585_wg_df['Day'] = pd.to_datetime(CMIP_SSP585_wg_df['Day'].astype(str)).dt.date

CMIP_SSP585_wg_df['Day'] = pd.to_datetime(CMIP_SSP585_wg_df['Day'])

CMIP_SSP585_wg_df = CMIP_SSP585_wg_df[CMIP_SSP585_wg_df['Day'] >= '2019-01-01']

# Display the first few rows of the DataFrame
CMIP_SSP585_wg_df


In [ ]:
# SSP585 - precipitation

In [ ]:
# Open the NetCDF file for reading
with nc.Dataset('./CMIP6_data/ssp585_prec_15-30.nc', 'r') as data:
    # Get a list of variable names
    variable_names = list(data.variables.keys())

# Print the variable names
print("Variables in the NetCDF file:")
for var_name in variable_names:
    print(var_name)


In [ ]:
# inspect dataset
CMIP_SSP585_prec_ds = nc.Dataset('./CMIP6_data/ssp585_prec_15-30.nc', 'r')
CMIP_SSP585_prec_ds

In [ ]:
# Define the latitude and longitude coordinates for New York JFK Airport
ny_jfk_lat = 40.6413
ny_jfk_lon = -73.7781

# Find the nearest latitude and longitude indices in the dataset
lat_values = CMIP_SSP585_prec_ds.variables['lat'][:]
lon_values = CMIP_SSP585_prec_ds.variables['lon'][:]
lat_index = abs(lat_values - ny_jfk_lat).argmin()
lon_index = abs(lon_values - ny_jfk_lon).argmin()

# Extract time and temperature data
time_data = CMIP_SSP585_prec_ds.variables['time'][:]
prec_data = CMIP_SSP585_prec_ds.variables['pr'][:, lat_index, lon_index]

# Convert the time data to a more understandable format
time_units = CMIP_SSP585_prec_ds.variables['time'].units
time_calendar = CMIP_SSP585_prec_ds.variables['time'].calendar
time_in_days = nc.num2date(time_data, units=time_units, calendar=time_calendar)

# Create a Pandas DataFrame
CMIP_SSP585_prec_df = pd.DataFrame({'Day': time_in_days, 'CMIP SSP585 Precipitation Rate': prec_data})

CMIP_SSP585_prec_df['Day'] = pd.to_datetime(CMIP_SSP585_prec_df['Day'].astype(str)).dt.date

CMIP_SSP585_prec_df['Day'] = pd.to_datetime(CMIP_SSP585_prec_df['Day'])

CMIP_SSP585_prec_df = CMIP_SSP585_prec_df[CMIP_SSP585_prec_df['Day'] >= '2019-01-01']

# Display the first few rows of the DataFrame
CMIP_SSP585_prec_df


In [ ]:
# Drop unnecessary column in dfs

In [ ]:
hist_mintemp_df

In [ ]:
hist_mintemp_df.drop('Hist Min Temperature', axis=1, inplace=True)
hist_mintemp_df

In [ ]:
CMIP_hist_mintemp_df

In [ ]:
CMIP_hist_mintemp_df.drop('CMIP Hist Temperature', axis=1, inplace=True)
CMIP_hist_mintemp_df

In [ ]:
# application of Quantile Delta Mapping

In [ ]:
# Print column names and index for the observation DataFrame
print("Observation DataFrame:")
print(hist_mintemp_df.columns)
print(hist_mintemp_df.index.name)  # Check the index name

# Print column names and index for the model output DataFrame
print("\nModel Output DataFrame:")
print(CMIP_hist_mintemp_df.columns)
print(CMIP_hist_mintemp_df.index.name)  # Check the index name

# Set the existing index as the index for the observation DataFrame
hist_mintemp_df.set_index(hist_mintemp_df.index, inplace=True)

# Set the existing index as the index for the model output DataFrame
CMIP_hist_mintemp_df.set_index('Day', inplace=True)

# Calculate quantiles for past observations
quantiles_observation = hist_mintemp_df['Hist Min Temperature (°C)'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

# Calculate quantiles for past model output
quantiles_model_output = CMIP_hist_mintemp_df['CMIP Hist Temperature (C°)'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

# Print the calculated quantiles
print("\nQuantiles for Past Observations:")
print(quantiles_observation)

print("\nQuantiles for Past Model Output:")
print(quantiles_model_output)


In [ ]:
# Calculate the deltas for each quantile
mt_deltas = quantiles_observation - quantiles_model_output

# Print the calculated deltas
print("Deltas:")
print(mt_deltas)


In [ ]:
mt_deltas = pd.DataFrame(mt_deltas)
mt_deltas

In [ ]:
# Values for the new row
new_values = -0.100355

# Adding a new row with index 1.0
mt_deltas.loc[1.0] = new_values

In [ ]:
mt_deltas

In [ ]:
# Calculate quantiles of future dataset

# Set the existing index as the index for the model output DataFrame
# CMIP_SSP126_mintemp_df.set_index('Day', inplace=True)

# Calculate quantiles for future model output
ssp126_mt_quantiles = CMIP_SSP126_mintemp_df['CMIP SSP126 Temperature (°C)'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

print("\nQuantiles for Future Model Output:")
print(ssp126_mt_quantiles)


In [ ]:
ssp126_mt_quantiles

In [ ]:
# tranform to pandas dataframe
ssp126_mt_quantiles = pd.DataFrame(ssp126_mt_quantiles)

In [ ]:
# Iterate over the rows of CMIP_SSP126_mintemp_df
for index, row in CMIP_SSP126_mintemp_df.iterrows():
    # Get the temperature value from CMIP_SSP126_mintemp_df
    temp_value = row['CMIP SSP126 Temperature (°C)']
    
    # Find the quantile index in ssp126_mt_quantiles where the temperature is smaller or equal
    quantile_index = ssp126_mt_quantiles[ssp126_mt_quantiles['CMIP SSP126 Temperature (°C)'] >= temp_value].index.min()
    
    # If there is a match, assign the quantile index to a new column in CMIP_SSP126_mintemp_df
    if pd.notna(quantile_index):
        CMIP_SSP126_mintemp_df.at[index, 'Quantile Index'] = quantile_index
    else:
        CMIP_SSP126_mintemp_df.at[index, 'Quantile Index'] = 1.0

# Display the modified CMIP_SSP126_mintemp_df
print(CMIP_SSP126_mintemp_df)


In [ ]:
# Iterate over the rows of CMIP_SSP126_mintemp_df
for index, row in CMIP_SSP126_mintemp_df.iterrows():
    # Get the quantile index from CMIP_SSP126_mintemp_df
    quantile_index = row['Quantile Index']
    
    # Check if the quantile index exists in mt_deltas
    if quantile_index in mt_deltas.index:
        # If it exists, assign the corresponding value to a new column in CMIP_SSP126_mintemp_df
        CMIP_SSP126_mintemp_df.at[index, 'Delta Value'] = mt_deltas.loc[quantile_index, 0]

# Display the modified CMIP_SSP126_mintemp_df
CMIP_SSP126_mintemp_df


In [ ]:
CMIP_SSP126_mintemp_df['Corrected CMIP SSP126 Temperature (°C)'] = CMIP_SSP126_mintemp_df['CMIP SSP126 Temperature (°C)'] + CMIP_SSP126_mintemp_df['Delta Value']

CMIP_SSP126_mintemp_df

In [ ]:
# Quantile Delta Mapping - SSP126 wind gust

In [ ]:
CMIP_hist_wind_df

In [ ]:
hist_wind_df

In [ ]:
CMIP_SSP126_wg_df

In [ ]:
# # Print column names and index for the observation DataFrame
# print("Observation DataFrame:")
# print(hist_mintemp_df.columns)
# print(hist_mintemp_df.index.name)  # Check the index name

# # Print column names and index for the model output DataFrame
# print("\nModel Output DataFrame:")
# print(CMIP_hist_mintemp_df.columns)
# print(CMIP_hist_mintemp_df.index.name)  # Check the index name

# Set the existing index as the index for the observation DataFrame
hist_wind_df.set_index('Date', inplace=True)

# Set the existing index as the index for the model output DataFrame
CMIP_hist_wind_df.set_index('Day', inplace=True)

# Calculate quantiles for past observations
wg_quantiles_observation = hist_wind_df['Hist Average Wind Gust'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

# Calculate quantiles for past model output
wg_quantiles_model_output = CMIP_hist_wind_df['CMIP Hist Wind'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

# Print the calculated quantiles
print("\nQuantiles for Past Observations:")
print(wg_quantiles_observation)

print("\nQuantiles for Past Model Output:")
print(wg_quantiles_model_output)


In [ ]:
# Calculate the deltas for each quantile
wg_deltas = wg_quantiles_observation - wg_quantiles_model_output

# Print the calculated deltas
print("Deltas:")
print(wg_deltas)


In [ ]:
hist_wg_delta = pd.DataFrame(wg_deltas)
hist_wg_delta

In [ ]:
# Calculate quantiles of future dataset

# Calculate quantiles for future model output
ssp126_wg_quantiles = CMIP_SSP126_wg_df['CMIP SSP126 Wind Gust'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

print("\nQuantiles for Future Model Output:")
print(ssp126_wg_quantiles)


In [ ]:
# transform to pandas dataframe

ssp126_wg_quantiles = pd.DataFrame(ssp126_wg_quantiles)

In [ ]:
# Iterate over the rows of CMIP_SSP126_mintemp_df
for index, row in CMIP_SSP126_wg_df.iterrows():
    # Get the wg value from CMIP_SSP126_wg_df
    wg_value = row['CMIP SSP126 Wind Gust']
    
    # Find the quantile index in ssp126_mt_quantiles where the temperature is smaller or equal
    quantile_index = ssp126_wg_quantiles[ssp126_wg_quantiles['CMIP SSP126 Wind Gust'] >= wg_value].index.min()
    
    # If there is a match, assign the quantile index to a new column in CMIP_SSP126_mintemp_df
    if pd.notna(quantile_index):
        CMIP_SSP126_wg_df.at[index, 'Quantile Index'] = quantile_index
    else:
        CMIP_SSP126_wg_df.at[index, 'Quantile Index'] = 1.0

# Display the modified CMIP_SSP126_mintemp_df
print(CMIP_SSP126_wg_df)


In [ ]:
# Iterate over the rows of CMIP_SSP126_wg_df
for index, row in CMIP_SSP126_wg_df.iterrows():
    # Get the quantile index from CMIP_SSP126_mintemp_df
    quantile_index = row['Quantile Index']
    
    # Check if the quantile index exists in mt_deltas
    if quantile_index in hist_wg_delta.index:
        # If it exists, assign the corresponding value to a new column in CMIP_SSP126_wg_df
        CMIP_SSP126_wg_df.at[index, 'Delta Value'] = hist_wg_delta.loc[quantile_index, 0]

# Display the modified CMIP_SSP126_wg_df
print(CMIP_SSP126_wg_df)


In [ ]:
CMIP_SSP126_wg_df['Corrected CMIP SSP126 Wind Gust'] = CMIP_SSP126_wg_df['CMIP SSP126 Wind Gust'] + CMIP_SSP126_wg_df['Delta Value']

CMIP_SSP126_wg_df

In [ ]:
# Delta Quantile Mapping - SSP126 - Precipitation

In [ ]:
hist_prec_df

In [ ]:
CMIP_hist_prec_df

In [ ]:
CMIP_SSP126_prec_df

In [ ]:
# # Print column names and index for the observation DataFrame
# print("Observation DataFrame:")
# print(hist_mintemp_df.columns)
# print(hist_mintemp_df.index.name)  # Check the index name

# # Print column names and index for the model output DataFrame
# print("\nModel Output DataFrame:")
# print(CMIP_hist_mintemp_df.columns)
# print(CMIP_hist_mintemp_df.index.name)  # Check the index name

# Set the existing index as the index for the observation DataFrame
hist_prec_df.set_index('Date', inplace=True)

# Set the existing index as the index for the model output DataFrame
CMIP_hist_prec_df.set_index('Day', inplace=True)

# Calculate quantiles for past observations
prec_quantiles_observation = hist_prec_df['Hist Average Precipitation'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

# Calculate quantiles for past model output
prec_quantiles_model_output = CMIP_hist_prec_df['CMIP Hist Precipitation'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

# Print the calculated quantiles
print("\nQuantiles for Past Observations:")
print(prec_quantiles_observation)

print("\nQuantiles for Past Model Output:")
print(prec_quantiles_model_output)


In [ ]:
# Calculate the deltas for each quantile
prec_deltas = prec_quantiles_observation - prec_quantiles_model_output

# Print the calculated deltas
print("Deltas:")
print(prec_deltas)


In [ ]:
prec_deltas = pd.DataFrame(prec_deltas)
prec_deltas

In [ ]:
# Calculate quantiles of future dataset

# Set the existing index as the index for the model output DataFrame
# CMIP_SSP126_mintemp_df.set_index('Day', inplace=True)

# Calculate quantiles for future model output
ssp126_prec_quantiles = CMIP_SSP126_prec_df['CMIP SSP126 Precipitation Rate'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

print("\nQuantiles for Future Model Output:")
print(ssp126_prec_quantiles)


In [ ]:
ssp126_prec_quantiles = pd.DataFrame(ssp126_prec_quantiles)

ssp126_prec_quantiles

In [ ]:
# Iterate over the rows of CMIP_SSP126_mintemp_df
for index, row in CMIP_SSP126_prec_df.iterrows():
    # Get the wg value from CMIP_SSP126_wg_df
    prec_value = row['CMIP SSP126 Precipitation Rate']
    
    # Find the quantile index in ssp126_mt_quantiles where the temperature is smaller or equal
    quantile_index = ssp126_prec_quantiles[ssp126_prec_quantiles['CMIP SSP126 Precipitation Rate'] >= prec_value].index.min()
    
    # If there is a match, assign the quantile index to a new column in CMIP_SSP126_mintemp_df
    if pd.notna(quantile_index):
        CMIP_SSP126_prec_df.at[index, 'Quantile Index'] = quantile_index
    else:
        CMIP_SSP126_prec_df.at[index, 'Quantile Index'] = 1.0

# Display the modified CMIP_SSP126_mintemp_df
print(CMIP_SSP126_prec_df)


In [ ]:
# Iterate over the rows of CMIP_SSP126_wg_df
for index, row in CMIP_SSP126_prec_df.iterrows():
    # Get the quantile index from CMIP_SSP126_mintemp_df
    quantile_index = row['Quantile Index']
    
    # Check if the quantile index exists in mt_deltas
    if quantile_index in prec_deltas.index:
        # If it exists, assign the corresponding value to a new column in CMIP_SSP126_wg_df
        CMIP_SSP126_prec_df.at[index, 'Delta Value'] = prec_deltas.loc[quantile_index, 0]

# Display the modified CMIP_SSP126_wg_df
print(CMIP_SSP126_prec_df)


In [ ]:
CMIP_SSP126_prec_df['Corrected CMIP SSP126 Precipitation Rate'] = CMIP_SSP126_prec_df['CMIP SSP126 Precipitation Rate'] + CMIP_SSP126_prec_df['Delta Value']

CMIP_SSP126_prec_df

In [ ]:
# SSP585 - Quantile Delta Mapping

In [ ]:
# Min Temp

In [ ]:
CMIP_SSP585_mintemp_df

In [ ]:
# Calculate quantiles of future dataset

# Set the existing index as the index for the model output DataFrame
# CMIP_SSP126_mintemp_df.set_index('Day', inplace=True)

# Calculate quantiles for future model output
ssp585_mt_quantiles = CMIP_SSP585_mintemp_df['CMIP SSP585 Temperature (°C)'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

print("\nQuantiles for Future Model Output:")
print(ssp585_mt_quantiles)


In [ ]:
ssp585_mt_quantiles = pd.DataFrame(ssp585_mt_quantiles)
ssp585_mt_quantiles

In [ ]:
# Iterate over the rows of CMIP_SSP126_mintemp_df
for index, row in CMIP_SSP585_mintemp_df.iterrows():
    # Get the temperature value from CMIP_SSP126_mintemp_df
    temp_value = row['CMIP SSP585 Temperature (°C)']
    
    # Find the quantile index in ssp126_mt_quantiles where the temperature is smaller or equal
    quantile_index = ssp585_mt_quantiles[ssp585_mt_quantiles['CMIP SSP585 Temperature (°C)'] >= temp_value].index.min()
    
    # If there is a match, assign the quantile index to a new column in CMIP_SSP126_mintemp_df
    if pd.notna(quantile_index):
        CMIP_SSP585_mintemp_df.at[index, 'Quantile Index'] = quantile_index
    else:
        CMIP_SSP585_mintemp_df.at[index, 'Quantile Index'] = 1.0

# Display the modified CMIP_SSP126_mintemp_df
print(CMIP_SSP585_mintemp_df)


In [ ]:
# Iterate over the rows of CMIP_SSP126_mintemp_df
for index, row in CMIP_SSP585_mintemp_df.iterrows():
    # Get the quantile index from CMIP_SSP126_mintemp_df
    quantile_index = row['Quantile Index']
    
    # Check if the quantile index exists in mt_deltas
    if quantile_index in mt_deltas.index:
        # If it exists, assign the corresponding value to a new column in CMIP_SSP126_mintemp_df
        CMIP_SSP585_mintemp_df.at[index, 'Delta Value'] = mt_deltas.loc[quantile_index, 0]

# Display the modified CMIP_SSP126_mintemp_df
print(CMIP_SSP585_mintemp_df)


In [ ]:
CMIP_SSP585_mintemp_df['Corrected CMIP SSP585 Temperature (°C)'] = CMIP_SSP585_mintemp_df['CMIP SSP585 Temperature (°C)'] + CMIP_SSP585_mintemp_df['Delta Value']

CMIP_SSP585_mintemp_df

In [ ]:
# SSP585 - Quantile Delta Mapping - Wind Gust

In [ ]:
CMIP_SSP585_wg_df

In [ ]:
# Calculate quantiles of future dataset

# Set the existing index as the index for the model output DataFrame
# CMIP_SSP126_mintemp_df.set_index('Day', inplace=True)

# Calculate quantiles for future model output
ssp585_wg_quantiles = CMIP_SSP585_wg_df['CMIP SSP585 Wind Gust'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

print("\nQuantiles for Future Model Output:")
print(ssp585_wg_quantiles)


In [ ]:
ssp585_wg_quantiles = pd.DataFrame(ssp585_wg_quantiles)

ssp585_wg_quantiles

In [ ]:
# Iterate over the rows of CMIP_SSP126_mintemp_df
for index, row in CMIP_SSP585_wg_df.iterrows():
    # Get the wg value from CMIP_SSP126_wg_df
    wg_value = row['CMIP SSP585 Wind Gust']
    
    # Find the quantile index in ssp126_mt_quantiles where the temperature is smaller or equal
    quantile_index = ssp585_wg_quantiles[ssp585_wg_quantiles['CMIP SSP585 Wind Gust'] >= wg_value].index.min()
    
    # If there is a match, assign the quantile index to a new column in CMIP_SSP126_mintemp_df
    if pd.notna(quantile_index):
        CMIP_SSP585_wg_df.at[index, 'Quantile Index'] = quantile_index
    else:
        CMIP_SSP585_wg_df.at[index, 'Quantile Index'] = 1.0

# Display the modified CMIP_SSP126_mintemp_df
print(CMIP_SSP585_wg_df)


In [ ]:
# Iterate over the rows of CMIP_SSP126_wg_df
for index, row in CMIP_SSP585_wg_df.iterrows():
    # Get the quantile index from CMIP_SSP126_mintemp_df
    quantile_index = row['Quantile Index']
    
    # Check if the quantile index exists in mt_deltas
    if quantile_index in hist_wg_delta.index:
        # If it exists, assign the corresponding value to a new column in CMIP_SSP126_wg_df
        CMIP_SSP585_wg_df.at[index, 'Delta Value'] = hist_wg_delta.loc[quantile_index, 0]

# Display the modified CMIP_SSP126_wg_df
print(CMIP_SSP585_wg_df)


In [ ]:
CMIP_SSP585_wg_df['Corrected CMIP SSP585 Wind Gust'] = CMIP_SSP585_wg_df['CMIP SSP585 Wind Gust'] + CMIP_SSP585_wg_df['Delta Value']

CMIP_SSP585_wg_df

In [ ]:
# SSP585 - Delta Quantile Mapping - Precipitation

In [ ]:
CMIP_SSP585_prec_df

In [ ]:
# Calculate quantiles of future dataset

# Calculate quantiles for future model output
ssp585_prec_quantiles = CMIP_SSP585_prec_df['CMIP SSP585 Precipitation Rate'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

print("\nQuantiles for Future Model Output:")
print(ssp585_prec_quantiles)


In [ ]:
ssp585_prec_quantiles = pd.DataFrame(ssp585_prec_quantiles)

ssp585_prec_quantiles

In [ ]:
# Iterate over the rows of CMIP_SSP126_mintemp_df
for index, row in CMIP_SSP585_prec_df.iterrows():
    # Get the wg value from CMIP_SSP126_wg_df
    prec_value = row['CMIP SSP585 Precipitation Rate']
    
    # Find the quantile index in ssp126_mt_quantiles where the temperature is smaller or equal
    quantile_index = ssp585_prec_quantiles[ssp585_prec_quantiles['CMIP SSP585 Precipitation Rate'] >= prec_value].index.min()
    
    # If there is a match, assign the quantile index to a new column in CMIP_SSP126_mintemp_df
    if pd.notna(quantile_index):
        CMIP_SSP585_prec_df.at[index, 'Quantile Index'] = quantile_index
    else:
        CMIP_SSP585_prec_df.at[index, 'Quantile Index'] = 1.0

# Display the modified CMIP_SSP126_mintemp_df
print(CMIP_SSP585_prec_df)


In [ ]:
# Iterate over the rows of CMIP_SSP126_wg_df
for index, row in CMIP_SSP585_prec_df.iterrows():
    # Get the quantile index from CMIP_SSP126_mintemp_df
    quantile_index = row['Quantile Index']
    
    # Check if the quantile index exists in mt_deltas
    if quantile_index in prec_deltas.index:
        # If it exists, assign the corresponding value to a new column in CMIP_SSP126_wg_df
        CMIP_SSP585_prec_df.at[index, 'Delta Value'] = prec_deltas.loc[quantile_index, 0]

# Display the modified CMIP_SSP126_wg_df
print(CMIP_SSP585_prec_df)


In [ ]:
CMIP_SSP585_prec_df['Corrected CMIP SSP585 Precipitation Rate'] = CMIP_SSP585_prec_df['CMIP SSP585 Precipitation Rate'] + CMIP_SSP585_prec_df['Delta Value']
CMIP_SSP585_prec_df

In [ ]:
# create df with corrected SSP126 data

In [ ]:
# Merge the dataframes on the 'FL_DATE' and 'Date' columns
merged_df = df_1.merge(df_2, left_on='FL_DATE', right_on='Date', how='left')

# Drop the 'Date' column from the merged dataframe if you don't need it
merged_df = merged_df.drop(columns=['Date'])

# Now, merged_df contains the Max Wind Gust and Average Wind Gust columns from df_2


In [ ]:
# Merge all corrected climate variable columns on the date

corrected_ssp126_df = CMIP_SSP126_mintemp_df.merge(CMIP_SSP126_wg_df, left_on='Day', right_on='Day', how='left')

corrected_ssp126_df = corrected_ssp126_df.drop(columns=['CMIP SSP126 Temperature (°C)', 'Quantile Index_x', 'Delta Value_x', 'CMIP SSP126 Wind Gust', 'Quantile Index_y', 'Delta Value_y'])     

corrected_ssp126_df = corrected_ssp126_df.merge(CMIP_SSP126_prec_df, left_on='Day', right_on='Day', how='left')

corrected_ssp126_df = corrected_ssp126_df.drop(columns=['CMIP SSP126 Precipitation Rate', 'Quantile Index', 'Delta Value'])     

corrected_ssp126_df 

In [ ]:
# create df with corrected SSP585 data

In [ ]:
corrected_ssp585_df = CMIP_SSP585_mintemp_df.merge(CMIP_SSP585_wg_df, left_on='Day', right_on='Day', how='left')

corrected_ssp585_df = corrected_ssp585_df.drop(columns=['CMIP SSP585 Temperature (°C)', 'Quantile Index_x', 'Delta Value_x', 'CMIP SSP585 Wind Gust', 'Quantile Index_y', 'Delta Value_y'])     

corrected_ssp585_df = corrected_ssp585_df.merge(CMIP_SSP585_prec_df, left_on='Day', right_on='Day', how='left')

corrected_ssp585_df = corrected_ssp585_df.drop(columns=['CMIP SSP585 Precipitation Rate', 'Quantile Index', 'Delta Value'])     

corrected_ssp585_df 

In [ ]:
# Create new df for each year from 2023 to 2030 with the same flight schedule as in 2022 but the future values

In [ ]:
df_2022 = df_1[df_1['FL_DATE'].dt.year == 2022]
df_2022

In [ ]:
columns_to_drop = ['ORIGIN', 'DEP_DELAY_NEW', 'CANCELLED', 'WEATHER_DELAY', 'NAS_DELAY', 'Delayed']
df_2022 = df_2022.drop(columns=columns_to_drop)

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column
df_2022['DOY'] = pd.to_datetime(df_2022['FL_DATE']).dt.dayofyear
df_2022 = df_2022.drop(columns=['FL_DATE'])  # Remove the original 'FL_DATE' column

In [ ]:
corrected_ssp126_df_2022 = corrected_ssp126_df[corrected_ssp126_df['Day'].dt.year == 2022]

In [ ]:
corrected_ssp126_df_2023 = corrected_ssp126_df[corrected_ssp126_df['Day'].dt.year == 2023]

In [ ]:
corrected_ssp126_df_2024 = corrected_ssp126_df[corrected_ssp126_df['Day'].dt.year == 2024]

In [ ]:
corrected_ssp126_df_2025 = corrected_ssp126_df[corrected_ssp126_df['Day'].dt.year == 2025]

In [ ]:
corrected_ssp126_df_2026 = corrected_ssp126_df[corrected_ssp126_df['Day'].dt.year == 2026]

In [ ]:
corrected_ssp126_df_2027 = corrected_ssp126_df[corrected_ssp126_df['Day'].dt.year == 2027]

In [ ]:
corrected_ssp126_df_2028 = corrected_ssp126_df[corrected_ssp126_df['Day'].dt.year == 2028]

In [ ]:
corrected_ssp126_df_2029 = corrected_ssp126_df[corrected_ssp126_df['Day'].dt.year == 2029]

In [ ]:
corrected_ssp126_df_2030 = corrected_ssp126_df[corrected_ssp126_df['Day'].dt.year == 2030]

In [ ]:
corrected_ssp126_df_2029

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp126_df_2030['DOY'] = pd.to_datetime(corrected_ssp126_df_2030['Day']).dt.dayofyear
corrected_ssp126_df_2030 = corrected_ssp126_df_2030.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp126_df_2029['DOY'] = pd.to_datetime(corrected_ssp126_df_2029['Day']).dt.dayofyear
corrected_ssp126_df_2029 = corrected_ssp126_df_2029.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp126_df_2028['DOY'] = pd.to_datetime(corrected_ssp126_df_2028['Day']).dt.dayofyear
corrected_ssp126_df_2028 = corrected_ssp126_df_2028.drop(columns=['Day'])

In [ ]:
# correct doy because 2028 is a leap year
corrected_ssp126_df_2028.loc[corrected_ssp126_df_2028['DOY'] >= 61, 'DOY'] -= 1

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp126_df_2027['DOY'] = pd.to_datetime(corrected_ssp126_df_2027['Day']).dt.dayofyear
corrected_ssp126_df_2027 = corrected_ssp126_df_2027.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp126_df_2026['DOY'] = pd.to_datetime(corrected_ssp126_df_2026['Day']).dt.dayofyear
corrected_ssp126_df_2026 = corrected_ssp126_df_2026.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp126_df_2025['DOY'] = pd.to_datetime(corrected_ssp126_df_2025['Day']).dt.dayofyear
corrected_ssp126_df_2025 = corrected_ssp126_df_2025.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp126_df_2024['DOY'] = pd.to_datetime(corrected_ssp126_df_2024['Day']).dt.dayofyear
corrected_ssp126_df_2024 = corrected_ssp126_df_2024.drop(columns=['Day'])

In [ ]:
# Correct DOY column because 2024 is a leap year
corrected_ssp126_df_2024.loc[corrected_ssp126_df_2024['DOY'] >= 61, 'DOY'] -= 1

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp126_df_2023['DOY'] = pd.to_datetime(corrected_ssp126_df_2023['Day']).dt.dayofyear
corrected_ssp126_df_2023 = corrected_ssp126_df_2023.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp126_df_2022['DOY'] = pd.to_datetime(corrected_ssp126_df_2022['Day']).dt.dayofyear
corrected_ssp126_df_2022 = corrected_ssp126_df_2022.drop(columns=['Day'])

In [ ]:
corrected_ssp126_df_2030 = df_2022.merge(corrected_ssp126_df_2030, on='DOY', how='left')

In [ ]:
corrected_ssp126_df_2029 = df_2022.merge(corrected_ssp126_df_2029, on='DOY', how='left')

In [ ]:
corrected_ssp126_df_2028 = df_2022.merge(corrected_ssp126_df_2028, on='DOY', how='left')

In [ ]:
corrected_ssp126_df_2027 = df_2022.merge(corrected_ssp126_df_2027, on='DOY', how='left')

In [ ]:
corrected_ssp126_df_2026 = df_2022.merge(corrected_ssp126_df_2026, on='DOY', how='left')

In [ ]:
corrected_ssp126_df_2025 = df_2022.merge(corrected_ssp126_df_2025, on='DOY', how='left')

In [ ]:
corrected_ssp126_df_2024 = df_2022.merge(corrected_ssp126_df_2024, on='DOY', how='left')

In [ ]:
corrected_ssp126_df_2023 = df_2022.merge(corrected_ssp126_df_2023, on='DOY', how='left')

In [ ]:
corrected_ssp126_df_2022 = df_2022.merge(corrected_ssp126_df_2022, on='DOY', how='left')

In [ ]:
corrected_ssp126_df_2030

In [ ]:
years = range(2022, 2031)

# Create a dictionary to map old column names to new column names
column_mapping = {'Corrected CMIP SSP126 Temperature (°C)': 'Min Temperature (°C)',
                  'Corrected CMIP SSP126 Wind Gust': 'Average Wind Gust',
                  'Corrected CMIP SSP126 Precipitation Rate': 'Average Precipitation'}

# Loop through the years and rename columns
for year in years:
    locals()[f'corrected_ssp126_df_{year}'].rename(columns=column_mapping, inplace=True)

In [ ]:
corrected_ssp585_df_2030 = corrected_ssp585_df[corrected_ssp585_df['Day'].dt.year == 2030]

In [ ]:
corrected_ssp585_df_2029 = corrected_ssp585_df[corrected_ssp585_df['Day'].dt.year == 2029]

In [ ]:
corrected_ssp585_df_2028 = corrected_ssp585_df[corrected_ssp585_df['Day'].dt.year == 2028]

In [ ]:
corrected_ssp585_df_2027 = corrected_ssp585_df[corrected_ssp585_df['Day'].dt.year == 2027]

In [ ]:
corrected_ssp585_df_2026 = corrected_ssp585_df[corrected_ssp585_df['Day'].dt.year == 2026]

In [ ]:
corrected_ssp585_df_2025 = corrected_ssp585_df[corrected_ssp585_df['Day'].dt.year == 2025]

In [ ]:
corrected_ssp585_df_2024 = corrected_ssp585_df[corrected_ssp585_df['Day'].dt.year == 2024]

In [ ]:
corrected_ssp585_df_2023 = corrected_ssp585_df[corrected_ssp585_df['Day'].dt.year == 2023]

In [ ]:
corrected_ssp585_df_2022 = corrected_ssp585_df[corrected_ssp585_df['Day'].dt.year == 2022]

In [ ]:
corrected_ssp585_df_2030

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp585_df_2030['DOY'] = pd.to_datetime(corrected_ssp585_df_2030['Day']).dt.dayofyear
corrected_ssp585_df_2030 = corrected_ssp585_df_2030.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp585_df_2029['DOY'] = pd.to_datetime(corrected_ssp585_df_2029['Day']).dt.dayofyear
corrected_ssp585_df_2029 = corrected_ssp585_df_2029.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp585_df_2028['DOY'] = pd.to_datetime(corrected_ssp585_df_2028['Day']).dt.dayofyear
corrected_ssp585_df_2028 = corrected_ssp585_df_2028.drop(columns=['Day'])

In [ ]:
# correct DOY column becasue 2028 is a leap year
corrected_ssp585_df_2028.loc[corrected_ssp585_df_2028['DOY'] >= 61, 'DOY'] -= 1

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp585_df_2027['DOY'] = pd.to_datetime(corrected_ssp585_df_2027['Day']).dt.dayofyear
corrected_ssp585_df_2027 = corrected_ssp585_df_2027.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp585_df_2026['DOY'] = pd.to_datetime(corrected_ssp585_df_2026['Day']).dt.dayofyear
corrected_ssp585_df_2026 = corrected_ssp585_df_2026.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp585_df_2025['DOY'] = pd.to_datetime(corrected_ssp585_df_2025['Day']).dt.dayofyear
corrected_ssp585_df_2025 = corrected_ssp585_df_2025.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp585_df_2024['DOY'] = pd.to_datetime(corrected_ssp585_df_2024['Day']).dt.dayofyear
corrected_ssp585_df_2024 = corrected_ssp585_df_2024.drop(columns=['Day'])

In [ ]:
# correct the DOY column because 2024 is a leap year
corrected_ssp585_df_2024.loc[corrected_ssp585_df_2024['DOY'] >= 61, 'DOY'] -= 1

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp585_df_2023['DOY'] = pd.to_datetime(corrected_ssp585_df_2023['Day']).dt.dayofyear
corrected_ssp585_df_2023 = corrected_ssp585_df_2023.drop(columns=['Day'])

In [ ]:
# Extract the Day of Year (DOY) from the 'FL_DATE' column and remove 'FL_DATE' column
corrected_ssp585_df_2022['DOY'] = pd.to_datetime(corrected_ssp585_df_2022['Day']).dt.dayofyear
corrected_ssp585_df_2022 = corrected_ssp585_df_2022.drop(columns=['Day'])

In [ ]:
corrected_ssp585_df_2030 = df_2022.merge(corrected_ssp585_df_2030, on='DOY', how='left')

In [ ]:
corrected_ssp585_df_2029 = df_2022.merge(corrected_ssp585_df_2029, on='DOY', how='left')

In [ ]:
corrected_ssp585_df_2028 = df_2022.merge(corrected_ssp585_df_2028, on='DOY', how='left')

In [ ]:
corrected_ssp585_df_2027 = df_2022.merge(corrected_ssp585_df_2027, on='DOY', how='left')

In [ ]:
corrected_ssp585_df_2026 = df_2022.merge(corrected_ssp585_df_2026, on='DOY', how='left')

In [ ]:
corrected_ssp585_df_2025 = df_2022.merge(corrected_ssp585_df_2025, on='DOY', how='left')

In [ ]:
corrected_ssp585_df_2024 = df_2022.merge(corrected_ssp585_df_2024, on='DOY', how='left')

In [ ]:
corrected_ssp585_df_2023 = df_2022.merge(corrected_ssp585_df_2023, on='DOY', how='left')

In [ ]:
corrected_ssp585_df_2022 = df_2022.merge(corrected_ssp585_df_2022, on='DOY', how='left')

In [ ]:
corrected_ssp585_df_2030

In [ ]:
years = range(2022, 2031)

# Create a dictionary to map old column names to new column names
column_mapping = {'Corrected CMIP SSP585 Temperature (°C)': 'Min Temperature (°C)', 'Corrected CMIP SSP585 Wind Gust': 'Average Wind Gust', 'Corrected CMIP SSP585 Precipitation Rate': 'Average Precipitation'}

# Rename the columns using the dictionary in a loop
for year in years:
    locals()[f'corrected_ssp585_df_{year}'].rename(columns=column_mapping, inplace=True)

corrected_ssp585_df_2030

In [ ]:
years = range(2022, 2031)

for year in years:
    locals()[f'corrected_ssp126_df_{year}'].drop(columns=['Weather_Delayed'], inplace=True)

In [ ]:
years = range(2022, 2031)

for year in years:
    locals()[f'corrected_ssp585_df_{year}'].drop(columns=['Weather_Delayed'], inplace=True)

In [ ]:
corrected_ssp585_df_2030

In [ ]:
# Make predictions on the new data with the gradient boost model - predict whether a flight is delayed or not

In [ ]:
# SSP 126

In [ ]:
new_X_126_30 = corrected_ssp126_df_2030[selected_features]

# Standardize numerical features
new_X_126_30[numerical_cols] = scaler.transform(new_X_126_30[numerical_cols])

# One-hot encode categorical features
new_X_encoded_126_30 = encoder.transform(new_X_126_30[categorical_cols])
new_feature_names_126_30 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_126_30 = pd.DataFrame(new_X_encoded_126_30, columns=new_feature_names_126_30)
new_X_126_30 = pd.concat([new_X_126_30.drop(columns=categorical_cols), new_X_encoded_126_30], axis=1)

In [ ]:
new_X_126_29 = corrected_ssp126_df_2029[selected_features]

# Standardize numerical features
new_X_126_29[numerical_cols] = scaler.transform(new_X_126_29[numerical_cols])

# One-hot encode categorical features
new_X_encoded_126_29 = encoder.transform(new_X_126_29[categorical_cols])
new_feature_names_126_29 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_126_29 = pd.DataFrame(new_X_encoded_126_29, columns=new_feature_names_126_29)
new_X_126_29 = pd.concat([new_X_126_29.drop(columns=categorical_cols), new_X_encoded_126_29], axis=1)

In [ ]:
new_X_126_28 = corrected_ssp126_df_2028[selected_features]

# Standardize numerical features
new_X_126_28[numerical_cols] = scaler.transform(new_X_126_28[numerical_cols])

# One-hot encode categorical features
new_X_encoded_126_28 = encoder.transform(new_X_126_28[categorical_cols])
new_feature_names_126_28 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_126_28 = pd.DataFrame(new_X_encoded_126_28, columns=new_feature_names_126_28)

new_X_126_28.reset_index(drop=True, inplace=True)
new_X_encoded_126_28.reset_index(drop=True, inplace=True)

new_X_126_28 = pd.concat([new_X_126_28.drop(columns=categorical_cols), new_X_encoded_126_28], axis=1)

In [ ]:
new_X_126_27 = corrected_ssp126_df_2027[selected_features]

# Standardize numerical features
new_X_126_27[numerical_cols] = scaler.transform(new_X_126_27[numerical_cols])

# One-hot encode categorical features
new_X_encoded_126_27 = encoder.transform(new_X_126_27[categorical_cols])
new_feature_names_126_27 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_126_27 = pd.DataFrame(new_X_encoded_126_27, columns=new_feature_names_126_27)
new_X_126_27 = pd.concat([new_X_126_27.drop(columns=categorical_cols), new_X_encoded_126_27], axis=1)

In [ ]:
new_X_126_26 = corrected_ssp126_df_2026[selected_features]

# Standardize numerical features
new_X_126_26[numerical_cols] = scaler.transform(new_X_126_26[numerical_cols])

# One-hot encode categorical features
new_X_encoded_126_26 = encoder.transform(new_X_126_26[categorical_cols])
new_feature_names_126_26 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_126_26 = pd.DataFrame(new_X_encoded_126_26, columns=new_feature_names_126_26)
new_X_126_26 = pd.concat([new_X_126_26.drop(columns=categorical_cols), new_X_encoded_126_26], axis=1)

In [ ]:
new_X_126_25 = corrected_ssp126_df_2025[selected_features]

# Standardize numerical features
new_X_126_25[numerical_cols] = scaler.transform(new_X_126_25[numerical_cols])

# One-hot encode categorical features
new_X_encoded_126_25 = encoder.transform(new_X_126_25[categorical_cols])
new_feature_names_126_25 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_126_25 = pd.DataFrame(new_X_encoded_126_25, columns=new_feature_names_126_25)
new_X_126_25 = pd.concat([new_X_126_25.drop(columns=categorical_cols), new_X_encoded_126_25], axis=1)

In [ ]:
new_X_126_24 = corrected_ssp126_df_2024[selected_features]

# Standardize numerical features
new_X_126_24[numerical_cols] = scaler.transform(new_X_126_24[numerical_cols])

# One-hot encode categorical features
new_X_encoded_126_24 = encoder.transform(new_X_126_24[categorical_cols])
new_feature_names_126_24 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_126_24 = pd.DataFrame(new_X_encoded_126_24, columns=new_feature_names_126_24)

new_X_126_24.reset_index(drop=True, inplace=True)
new_X_encoded_126_24.reset_index(drop=True, inplace=True)

new_X_126_24 = pd.concat([new_X_126_24.drop(columns=categorical_cols), new_X_encoded_126_24], axis=1)

In [ ]:
new_X_126_23 = corrected_ssp126_df_2023[selected_features]

# Standardize numerical features
new_X_126_23[numerical_cols] = scaler.transform(new_X_126_23[numerical_cols])

# One-hot encode categorical features
new_X_encoded_126_23 = encoder.transform(new_X_126_23[categorical_cols])
new_feature_names_126_23 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_126_23 = pd.DataFrame(new_X_encoded_126_23, columns=new_feature_names_126_23)
new_X_126_23 = pd.concat([new_X_126_23.drop(columns=categorical_cols), new_X_encoded_126_23], axis=1)

In [ ]:
new_X_126_22 = corrected_ssp126_df_2022[selected_features]

# Standardize numerical features
new_X_126_22[numerical_cols] = scaler.transform(new_X_126_22[numerical_cols])

# One-hot encode categorical features
new_X_encoded_126_22 = encoder.transform(new_X_126_22[categorical_cols])
new_feature_names_126_22 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_126_22 = pd.DataFrame(new_X_encoded_126_22, columns=new_feature_names_126_22)
new_X_126_22 = pd.concat([new_X_126_22.drop(columns=categorical_cols), new_X_encoded_126_22], axis=1)

In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2030['Predicted_Label'] = clf.predict(new_X_126_30)

# If you want probability scores as well
corrected_ssp126_df_2030['Probability_Score'] = clf.predict_proba(new_X_126_30)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2029['Predicted_Label'] = clf.predict(new_X_126_29)

# If you want probability scores as well
corrected_ssp126_df_2029['Probability_Score'] = clf.predict_proba(new_X_126_29)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2028['Predicted_Label'] = clf.predict(new_X_126_28)

# If you want probability scores as well
corrected_ssp126_df_2028['Probability_Score'] = clf.predict_proba(new_X_126_28)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2027['Predicted_Label'] = clf.predict(new_X_126_27)

# If you want probability scores as well
corrected_ssp126_df_2027['Probability_Score'] = clf.predict_proba(new_X_126_27)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2026['Predicted_Label'] = clf.predict(new_X_126_26)

# If you want probability scores as well
corrected_ssp126_df_2026['Probability_Score'] = clf.predict_proba(new_X_126_26)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2025['Predicted_Label'] = clf.predict(new_X_126_25)

# If you want probability scores as well
corrected_ssp126_df_2025['Probability_Score'] = clf.predict_proba(new_X_126_25)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2024['Predicted_Label'] = clf.predict(new_X_126_24)

# If you want probability scores as well
corrected_ssp126_df_2024['Probability_Score'] = clf.predict_proba(new_X_126_24)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2023['Predicted_Label'] = clf.predict(new_X_126_23)

# If you want probability scores as well
corrected_ssp126_df_2023['Probability_Score'] = clf.predict_proba(new_X_126_23)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2022['Predicted_Label'] = clf.predict(new_X_126_22)

# If you want probability scores as well
corrected_ssp126_df_2022['Probability_Score'] = clf.predict_proba(new_X_126_22)[:, 1]


In [ ]:
corrected_ssp126_df_2022

In [ ]:
corrected_ssp126_df_2030['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp126_df_2029['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp126_df_2028['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp126_df_2027['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp126_df_2026['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp126_df_2025['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp126_df_2024['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp126_df_2023['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp126_df_2022['Predicted_Label'].value_counts()

In [ ]:
# SSP 585

In [ ]:
new_X_585_30 = corrected_ssp585_df_2030[selected_features]

# Standardize numerical features
new_X_585_30[numerical_cols] = scaler.transform(new_X_585_30[numerical_cols])

# One-hot encode categorical features
new_X_encoded_585_30 = encoder.transform(new_X_585_30[categorical_cols])
new_feature_names_585_30 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_585_30 = pd.DataFrame(new_X_encoded_585_30, columns=new_feature_names_585_30)
new_X_585_30 = pd.concat([new_X_585_30.drop(columns=categorical_cols), new_X_encoded_585_30], axis=1)

In [ ]:
new_X_585_29 = corrected_ssp585_df_2029[selected_features]

# Standardize numerical features
new_X_585_29[numerical_cols] = scaler.transform(new_X_585_29[numerical_cols])

# One-hot encode categorical features
new_X_encoded_585_29 = encoder.transform(new_X_585_29[categorical_cols])
new_feature_names_585_29 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_585_29 = pd.DataFrame(new_X_encoded_585_29, columns=new_feature_names_585_29)
new_X_585_29 = pd.concat([new_X_585_29.drop(columns=categorical_cols), new_X_encoded_585_29], axis=1)

In [ ]:
new_X_585_28 = corrected_ssp585_df_2028[selected_features]

# Standardize numerical features
new_X_585_28[numerical_cols] = scaler.transform(new_X_585_28[numerical_cols])

# One-hot encode categorical features
new_X_encoded_585_28 = encoder.transform(new_X_585_28[categorical_cols])
new_feature_names_585_28 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_585_28 = pd.DataFrame(new_X_encoded_585_28, columns=new_feature_names_585_28)
new_X_585_28 = pd.concat([new_X_585_28.drop(columns=categorical_cols), new_X_encoded_585_28], axis=1)

In [ ]:
new_X_585_27 = corrected_ssp585_df_2027[selected_features]

# Standardize numerical features
new_X_585_27[numerical_cols] = scaler.transform(new_X_585_27[numerical_cols])

# One-hot encode categorical features
new_X_encoded_585_27 = encoder.transform(new_X_585_27[categorical_cols])
new_feature_names_585_27 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_585_27 = pd.DataFrame(new_X_encoded_585_27, columns=new_feature_names_585_27)
new_X_585_27 = pd.concat([new_X_585_27.drop(columns=categorical_cols), new_X_encoded_585_27], axis=1)

In [ ]:
new_X_585_26 = corrected_ssp585_df_2026[selected_features]

# Standardize numerical features
new_X_585_26[numerical_cols] = scaler.transform(new_X_585_26[numerical_cols])

# One-hot encode categorical features
new_X_encoded_585_26 = encoder.transform(new_X_585_26[categorical_cols])
new_feature_names_585_26 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_585_26 = pd.DataFrame(new_X_encoded_585_26, columns=new_feature_names_585_26)
new_X_585_26 = pd.concat([new_X_585_26.drop(columns=categorical_cols), new_X_encoded_585_26], axis=1)

In [ ]:
new_X_585_25 = corrected_ssp585_df_2025[selected_features]

# Standardize numerical features
new_X_585_25[numerical_cols] = scaler.transform(new_X_585_25[numerical_cols])

# One-hot encode categorical features
new_X_encoded_585_25 = encoder.transform(new_X_585_25[categorical_cols])
new_feature_names_585_25 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_585_25 = pd.DataFrame(new_X_encoded_585_25, columns=new_feature_names_585_25)
new_X_585_25 = pd.concat([new_X_585_25.drop(columns=categorical_cols), new_X_encoded_585_25], axis=1)

In [ ]:
new_X_585_24 = corrected_ssp585_df_2024[selected_features]

# Standardize numerical features
new_X_585_24[numerical_cols] = scaler.transform(new_X_585_24[numerical_cols])

# One-hot encode categorical features
new_X_encoded_585_24 = encoder.transform(new_X_585_24[categorical_cols])
new_feature_names_585_24 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_585_24 = pd.DataFrame(new_X_encoded_585_24, columns=new_feature_names_585_24)
new_X_585_24 = pd.concat([new_X_585_24.drop(columns=categorical_cols), new_X_encoded_585_24], axis=1)

In [ ]:
new_X_585_23 = corrected_ssp585_df_2023[selected_features]

# Standardize numerical features
new_X_585_23[numerical_cols] = scaler.transform(new_X_585_23[numerical_cols])

# One-hot encode categorical features
new_X_encoded_585_23 = encoder.transform(new_X_585_23[categorical_cols])
new_feature_names_585_23 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_585_23 = pd.DataFrame(new_X_encoded_585_23, columns=new_feature_names_585_23)
new_X_585_23 = pd.concat([new_X_585_23.drop(columns=categorical_cols), new_X_encoded_585_23], axis=1)

In [ ]:
new_X_585_22 = corrected_ssp585_df_2022[selected_features]

# Standardize numerical features
new_X_585_22[numerical_cols] = scaler.transform(new_X_585_22[numerical_cols])

# One-hot encode categorical features
new_X_encoded_585_22 = encoder.transform(new_X_585_22[categorical_cols])
new_feature_names_585_22 = encoder.get_feature_names_out(input_features=categorical_cols)
new_X_encoded_585_22 = pd.DataFrame(new_X_encoded_585_22, columns=new_feature_names_585_22)
new_X_585_22 = pd.concat([new_X_585_22.drop(columns=categorical_cols), new_X_encoded_585_22], axis=1)

In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2030['Predicted_Label'] = clf.predict(new_X_585_30)

# If you want probability scores as well
corrected_ssp585_df_2030['Probability_Score'] = clf.predict_proba(new_X_585_30)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2029['Predicted_Label'] = clf.predict(new_X_585_29)

# If you want probability scores as well
corrected_ssp585_df_2029['Probability_Score'] = clf.predict_proba(new_X_585_29)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2028['Predicted_Label'] = clf.predict(new_X_585_28)

# If you want probability scores as well
corrected_ssp585_df_2028['Probability_Score'] = clf.predict_proba(new_X_585_28)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2027['Predicted_Label'] = clf.predict(new_X_585_27)

# If you want probability scores as well
corrected_ssp585_df_2027['Probability_Score'] = clf.predict_proba(new_X_585_27)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2026['Predicted_Label'] = clf.predict(new_X_585_26)

# If you want probability scores as well
corrected_ssp585_df_2026['Probability_Score'] = clf.predict_proba(new_X_585_26)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2025['Predicted_Label'] = clf.predict(new_X_585_25)

# If you want probability scores as well
corrected_ssp585_df_2025['Probability_Score'] = clf.predict_proba(new_X_585_25)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2024['Predicted_Label'] = clf.predict(new_X_585_24)

# If you want probability scores as well
corrected_ssp585_df_2024['Probability_Score'] = clf.predict_proba(new_X_585_24)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2023['Predicted_Label'] = clf.predict(new_X_585_23)

# If you want probability scores as well
corrected_ssp585_df_2023['Probability_Score'] = clf.predict_proba(new_X_585_23)[:, 1]


In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2022['Predicted_Label'] = clf.predict(new_X_585_22)

# If you want probability scores as well
corrected_ssp585_df_2022['Probability_Score'] = clf.predict_proba(new_X_585_22)[:, 1]


In [ ]:
corrected_ssp585_df_2030['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp585_df_2029['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp585_df_2028['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp585_df_2027['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp585_df_2026['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp585_df_2025['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp585_df_2024['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp585_df_2023['Predicted_Label'].value_counts()

In [ ]:
corrected_ssp585_df_2022['Predicted_Label'].value_counts()

In [ ]:
# Add the predictions for the delay cost - regression model

In [ ]:
corrected_ssp126_df_2022

In [ ]:
for year in range(2022, 2031):
    locals()[f'corrected_ssp126_df_{year}'].rename(columns={'Predicted_Label': 'Weather_Delayed'}, inplace=True)

In [ ]:
for year in range(2022, 2031):
    locals()[f'corrected_ssp585_df_{year}'].rename(columns={'Predicted_Label': 'Weather_Delayed'}, inplace=True)

In [ ]:
corrected_ssp585_df_2030

In [ ]:
# predictions

In [ ]:
X_new_cost_ssp126_22 = corrected_ssp126_df_2022[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp126_22[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp126_22[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp126_22 = encoder_reg.transform(X_new_cost_ssp126_22[categorical_cols_reg])
X_encoded_new_cost_ssp126_22 = pd.DataFrame(X_encoded_new_cost_ssp126_22, columns=feature_names_reg)
X_new_cost_ssp126_22 = pd.concat([X_new_cost_ssp126_22.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp126_22], axis=1)

In [ ]:
X_new_cost_ssp126_23 = corrected_ssp126_df_2023[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp126_23[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp126_23[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp126_23 = encoder_reg.transform(X_new_cost_ssp126_23[categorical_cols_reg])
X_encoded_new_cost_ssp126_23 = pd.DataFrame(X_encoded_new_cost_ssp126_23, columns=feature_names_reg)
X_new_cost_ssp126_23 = pd.concat([X_new_cost_ssp126_23.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp126_23], axis=1)

In [ ]:
X_new_cost_ssp126_24 = corrected_ssp126_df_2024[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp126_24[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp126_24[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp126_24 = encoder_reg.transform(X_new_cost_ssp126_24[categorical_cols_reg])
X_encoded_new_cost_ssp126_24 = pd.DataFrame(X_encoded_new_cost_ssp126_24, columns=feature_names_reg)
X_new_cost_ssp126_24 = pd.concat([X_new_cost_ssp126_24.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp126_24], axis=1)

In [ ]:
X_new_cost_ssp126_25 = corrected_ssp126_df_2025[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp126_25[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp126_25[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp126_25 = encoder_reg.transform(X_new_cost_ssp126_25[categorical_cols_reg])
X_encoded_new_cost_ssp126_25 = pd.DataFrame(X_encoded_new_cost_ssp126_25, columns=feature_names_reg)
X_new_cost_ssp126_25 = pd.concat([X_new_cost_ssp126_25.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp126_25], axis=1)

In [ ]:
X_new_cost_ssp126_26 = corrected_ssp126_df_2026[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp126_26[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp126_26[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp126_26 = encoder_reg.transform(X_new_cost_ssp126_26[categorical_cols_reg])
X_encoded_new_cost_ssp126_26 = pd.DataFrame(X_encoded_new_cost_ssp126_26, columns=feature_names_reg)
X_new_cost_ssp126_26 = pd.concat([X_new_cost_ssp126_26.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp126_26], axis=1)

In [ ]:
X_new_cost_ssp126_27 = corrected_ssp126_df_2027[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp126_27[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp126_27[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp126_27 = encoder_reg.transform(X_new_cost_ssp126_27[categorical_cols_reg])
X_encoded_new_cost_ssp126_27 = pd.DataFrame(X_encoded_new_cost_ssp126_27, columns=feature_names_reg)
X_new_cost_ssp126_27 = pd.concat([X_new_cost_ssp126_27.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp126_27], axis=1)

In [ ]:
X_new_cost_ssp126_28 = corrected_ssp126_df_2028[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp126_28[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp126_28[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp126_28 = encoder_reg.transform(X_new_cost_ssp126_28[categorical_cols_reg])
X_encoded_new_cost_ssp126_28 = pd.DataFrame(X_encoded_new_cost_ssp126_28, columns=feature_names_reg)
X_new_cost_ssp126_28 = pd.concat([X_new_cost_ssp126_28.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp126_28], axis=1)

In [ ]:
X_new_cost_ssp126_29 = corrected_ssp126_df_2029[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp126_29[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp126_29[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp126_29 = encoder_reg.transform(X_new_cost_ssp126_29[categorical_cols_reg])
X_encoded_new_cost_ssp126_29 = pd.DataFrame(X_encoded_new_cost_ssp126_29, columns=feature_names_reg)
X_new_cost_ssp126_29 = pd.concat([X_new_cost_ssp126_29.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp126_29], axis=1)

In [ ]:
X_new_cost_ssp126_30 = corrected_ssp126_df_2030[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp126_30[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp126_30[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp126_30 = encoder_reg.transform(X_new_cost_ssp126_30[categorical_cols_reg])
X_encoded_new_cost_ssp126_30 = pd.DataFrame(X_encoded_new_cost_ssp126_30, columns=feature_names_reg)
X_new_cost_ssp126_30 = pd.concat([X_new_cost_ssp126_30.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp126_30], axis=1)

In [ ]:
# 585 cost prediction

In [ ]:
X_new_cost_ssp585_22 = corrected_ssp585_df_2022[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp585_22[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp585_22[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp585_22 = encoder_reg.transform(X_new_cost_ssp585_22[categorical_cols_reg])
X_encoded_new_cost_ssp585_22 = pd.DataFrame(X_encoded_new_cost_ssp585_22, columns=feature_names_reg)
X_new_cost_ssp585_22 = pd.concat([X_new_cost_ssp585_22.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp585_22], axis=1)

In [ ]:
X_new_cost_ssp585_23 = corrected_ssp585_df_2023[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp585_23[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp585_23[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp585_23 = encoder_reg.transform(X_new_cost_ssp585_23[categorical_cols_reg])
X_encoded_new_cost_ssp585_23 = pd.DataFrame(X_encoded_new_cost_ssp585_23, columns=feature_names_reg)
X_new_cost_ssp585_23 = pd.concat([X_new_cost_ssp585_23.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp585_23], axis=1)

In [ ]:
X_new_cost_ssp585_24 = corrected_ssp585_df_2024[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp585_24[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp585_24[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp585_24 = encoder_reg.transform(X_new_cost_ssp585_24[categorical_cols_reg])
X_encoded_new_cost_ssp585_24 = pd.DataFrame(X_encoded_new_cost_ssp585_24, columns=feature_names_reg)
X_new_cost_ssp585_24 = pd.concat([X_new_cost_ssp585_24.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp585_24], axis=1)

In [ ]:
X_new_cost_ssp585_25 = corrected_ssp585_df_2025[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp585_25[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp585_25[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp585_25 = encoder_reg.transform(X_new_cost_ssp585_25[categorical_cols_reg])
X_encoded_new_cost_ssp585_25 = pd.DataFrame(X_encoded_new_cost_ssp585_25, columns=feature_names_reg)
X_new_cost_ssp585_25 = pd.concat([X_new_cost_ssp585_25.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp585_25], axis=1)

In [ ]:
X_new_cost_ssp585_26 = corrected_ssp585_df_2026[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp585_26[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp585_26[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp585_26 = encoder_reg.transform(X_new_cost_ssp585_26[categorical_cols_reg])
X_encoded_new_cost_ssp585_26 = pd.DataFrame(X_encoded_new_cost_ssp585_26, columns=feature_names_reg)
X_new_cost_ssp585_26 = pd.concat([X_new_cost_ssp585_26.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp585_26], axis=1)

In [ ]:
X_new_cost_ssp585_27 = corrected_ssp585_df_2027[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp585_27[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp585_27[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp585_27 = encoder_reg.transform(X_new_cost_ssp585_27[categorical_cols_reg])
X_encoded_new_cost_ssp585_27 = pd.DataFrame(X_encoded_new_cost_ssp585_27, columns=feature_names_reg)
X_new_cost_ssp585_27 = pd.concat([X_new_cost_ssp585_27.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp585_27], axis=1)

In [ ]:
X_new_cost_ssp585_28 = corrected_ssp585_df_2028[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp585_28[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp585_28[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp585_28 = encoder_reg.transform(X_new_cost_ssp585_28[categorical_cols_reg])
X_encoded_new_cost_ssp585_28 = pd.DataFrame(X_encoded_new_cost_ssp585_28, columns=feature_names_reg)
X_new_cost_ssp585_28 = pd.concat([X_new_cost_ssp585_28.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp585_28], axis=1)

In [ ]:
X_new_cost_ssp585_29 = corrected_ssp585_df_2029[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp585_29[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp585_29[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp585_29 = encoder_reg.transform(X_new_cost_ssp585_29[categorical_cols_reg])
X_encoded_new_cost_ssp585_29 = pd.DataFrame(X_encoded_new_cost_ssp585_29, columns=feature_names_reg)
X_new_cost_ssp585_29 = pd.concat([X_new_cost_ssp585_29.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp585_29], axis=1)

In [ ]:
X_new_cost_ssp585_30 = corrected_ssp585_df_2030[regression_features]

# Standardize numerical features for regression
X_new_cost_ssp585_30[numerical_cols_reg] = scaler_reg.transform(X_new_cost_ssp585_30[numerical_cols_reg])

# Handle categorical features with one-hot encoding for regression
X_encoded_new_cost_ssp585_30 = encoder_reg.transform(X_new_cost_ssp585_30[categorical_cols_reg])
X_encoded_new_cost_ssp585_30 = pd.DataFrame(X_encoded_new_cost_ssp585_30, columns=feature_names_reg)
X_new_cost_ssp585_30 = pd.concat([X_new_cost_ssp585_30.drop(columns=categorical_cols_reg), X_encoded_new_cost_ssp585_30], axis=1)

In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2022['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp126_22)
corrected_ssp126_df_2022

In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2023['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp126_23)

In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2024['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp126_24)

In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2025['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp126_25)

In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2026['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp126_26)

In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2027['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp126_27)

In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2028['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp126_28)

In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2029['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp126_29)

In [ ]:
# Add predictions to dataset
corrected_ssp126_df_2030['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp126_30)

In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2022['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp126_22)

In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2023['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp585_23)

In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2024['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp585_24)

In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2025['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp585_25)

In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2026['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp585_26)

In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2027['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp585_27)

In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2028['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp585_28)

In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2029['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp585_29)

In [ ]:
# Add predictions to dataset
corrected_ssp585_df_2030['Predicted_Delay_Cost'] = regressor.predict(X_new_cost_ssp585_30)

In [ ]:
corrected_ssp585_df_2030

In [ ]:
# Results

In [ ]:
# Yearly predictions

In [ ]:
# List to store the data for the new dataframe
data = []

# Iterate through each year's dataframe
for year in range(2023, 2031):
    # Get the current dataframe
    current_df = globals()[f'corrected_ssp126_df_{year}']

    # Count occurrences of value 1 in "Predicted_Label_2" column
    weather_delays_count = current_df['Weather_Delayed'].eq(1).sum()

    # Append data to the list
    data.append({'year': year, 'weather_delays_126': weather_delays_count})

# Create a new dataframe from the list
new_table_df = pd.DataFrame(data)

# Print or use the new dataframe as needed
print(new_table_df)


In [ ]:
# Set the float_format to display full decimal representation rounded to two decimal places
pd.set_option('display.float_format', '{:.2f}'.format)

# List to store the data for the new dataframe
data_2 = []

# Iterate through each year's dataframe
for year in range(2023, 2031):
    # Get the current dataframe
    current_df = globals()[f'corrected_ssp126_df_{year}']

    # Sum values in the "Predicted_Delay_Cost" column
    total_delay_cost = current_df['Predicted_Delay_Cost'].sum()

    # Append data to the list
    data_2.append({'year': year, 'Total Delay Cost': total_delay_cost})

# Create a new dataframe from the list
new_table_df_cost = pd.DataFrame(data_2)

# Print or use the new dataframe as needed
print(new_table_df_cost)


In [ ]:
# List to store the data for the new dataframe
data_585 = []

# Iterate through each year's dataframe
for year in range(2023, 2031):
    # Get the current dataframe
    current_df_585 = globals()[f'corrected_ssp585_df_{year}']

    # Count occurrences of value 1 in "Predicted_Label_2" column
    weather_delays_count = current_df_585['Weather_Delayed'].eq(1).sum()

    # Append data to the list
    data_585.append({'year': year, 'weather_delays_585': weather_delays_count})

# Create a new dataframe from the list
new_table_df_585 = pd.DataFrame(data_585)

# Print or use the new dataframe as needed
print(new_table_df_585)


In [ ]:
# Set the float_format to display full decimal representation rounded to two decimal places
pd.set_option('display.float_format', '{:.2f}'.format)

# List to store the data for the new dataframe
data_585_2 = []

# Iterate through each year's dataframe
for year in range(2023, 2031):
    # Get the current dataframe
    current_df_585 = globals()[f'corrected_ssp585_df_{year}']

    # Sum values in the "Predicted_Delay_Cost" column
    total_delay_cost_585 = current_df_585['Predicted_Delay_Cost'].sum()

    # Append data to the list
    data_585_2.append({'year': year, 'Total Delay Cost': total_delay_cost_585})

# Create a new dataframe from the list
new_table_df_cost_585 = pd.DataFrame(data_585_2)

# Print or use the new dataframe as needed
print(new_table_df_cost_585)


In [ ]:
new_table_df = new_table_df.merge(new_table_df_585, on='year', how='left')

In [ ]:
new_table_df

In [ ]:
new_table_df_cost = new_table_df_cost.merge(new_table_df_cost_585, on='year', how='left')

In [ ]:
new_table_df_cost

In [ ]:
# Define a dictionary with the old and new column names
column_mapping = {'Total Delay Cost_x': 'Delay Cost 126', 'Total Delay Cost_y': 'Delay Cost 585'}

# Use the rename method to rename the columns
new_table_df_cost.rename(columns=column_mapping, inplace=True)

In [ ]:
new_table_df_cost

In [ ]:
merged_df_5.columns

In [ ]:
# Extract year from the date
merged_df_5['year'] = merged_df_5['FL_DATE'].dt.year

# Group by year and calculate the frequency of delays
result_df = merged_df_5.groupby('year')['Weather_Delayed'].sum().reset_index()
result_df.rename(columns={'Weather_Delayed': 'weather_delays_historical'}, inplace=True)

In [ ]:
result_df = result_df[result_df['year'] <= 2022]
result_df

In [ ]:
# Extract year from the date
merged_df_5['year'] = merged_df_5['FL_DATE'].dt.year

# Group by year and calculate the frequency of delays
result_df_cost = merged_df_5.groupby('year')['Estimated_Weather_Delay_Cost'].sum().reset_index()
result_df_cost.rename(columns={'Estimated_Weather_Delay_Cost': 'Delay Cost Historical'}, inplace=True)

In [ ]:
result_df_cost

In [ ]:
# Merging past and future delays
delays_year_df = pd.merge(result_df, new_table_df, on='year', how='outer')
delays_year_df

In [ ]:
# Calculate mean and values for the 1st standard deviation
columns_to_calculate = ['weather_delays_126', 'weather_delays_585']

for column in columns_to_calculate:
    # Exclude NaN values
    non_nan_values = delays_year_df[column].dropna()

    # Calculate mean
    mean_value = non_nan_values.mean()

    # Calculate 1st standard deviation
    std_dev_value = non_nan_values.std()

    # Calculate values for 1st standard deviation
    lower_bound = mean_value - std_dev_value
    upper_bound = mean_value + std_dev_value

    print(f"Column: {column}")
    print(f"Mean: {mean_value}")
    print(f"1st Standard Deviation: {std_dev_value}")
    print(f"Lower Bound: {lower_bound}")
    print(f"Upper Bound: {upper_bound}")
    print()

In [ ]:
# Merging past and future delay costs
cost_year_df = pd.merge(result_df_cost, new_table_df_cost, on='year', how='outer')
cost_year_df

In [ ]:
# Replace the value in the "Delay Cost Historical" column for the year 2023 with NaN
cost_year_df.loc[cost_year_df['year'] == 2023, 'Delay Cost Historical'] = np.nan
cost_year_df

In [ ]:
# Define a function to calculate mean, std, and bounds
def calculate_stats(column):
    non_nan_values = column.dropna()
    mean_value = non_nan_values.mean()
    std_value = non_nan_values.std()
    lower_bound = mean_value - std_value
    upper_bound = mean_value + std_value
    return mean_value, std_value, lower_bound, upper_bound

# Calculate stats for Delay Cost 126
mean_126, std_126, lower_126, upper_126 = calculate_stats(cost_year_df['Delay Cost 126'])

# Calculate stats for Delay Cost 585
mean_585, std_585, lower_585, upper_585 = calculate_stats(cost_year_df['Delay Cost 585'])

# Print the results
print("Delay Cost 126:")
print(f"Mean: {mean_126}")
print(f"1st Std Dev: {std_126}")
print(f"Lower Bound: {lower_126}")
print(f"Upper Bound: {upper_126}")
print("\nDelay Cost 585:")
print(f"Mean: {mean_585}")
print(f"1st Std Dev: {std_585}")
print(f"Lower Bound: {lower_585}")
print(f"Upper Bound: {upper_585}")

In [ ]:
# Calculate number of ticket that need to be sold to cover the cost - SSP 1.26
43092860.025846064/416.17 

In [ ]:
# Calculate number of ticket that need to be sold to cover the cost - SSP 5.85
41994056.25977469/416.17 

In [ ]:
import matplotlib.pyplot as plt

# ... (Your existing code for data processing)

# Plotting
plt.figure(figsize=(10, 6))

# Plot historical values
plt.plot(delays_year_df['year'], delays_year_df['weather_delays_historical'], color='gray', linestyle='-', marker='o', label='Historical')

# Plot average lines and spreads for non-NaN values
plt.plot(delays_year_df.loc[10:, 'year'], [avg_delay_585] * len(delays_year_df.loc[10:]), color='b', linestyle='--', label='Avg Delay')
plt.fill_between(delays_year_df.loc[10:, 'year'], avg_delay_585 - std_delay_585, avg_delay_585 + std_delay_585, where=~delays_year_df['weather_delays_585'].isna()[10:], color='lightblue', alpha=0.5, label='Spread')

# Plot individual data points
plt.scatter(delays_year_df.loc[10:, 'year'], delays_year_df.loc[10:, 'weather_delays_585'], color='darkblue', label='Data', marker='o', s=50)

# Plot error bars for individual data points
plt.errorbar(delays_year_df.loc[10:, 'year'], delays_year_df.loc[10:, 'weather_delays_585'], yerr=std_delay_585, linestyle='', color='darkblue', label='Std Dev', capsize=4)

plt.xlabel('Year [-]', fontname='Times New Roman', fontsize=16)
plt.ylabel('Number of Delays [-]', fontname='Times New Roman', fontsize=16)
plt.legend(loc='upper left', title=None)
plt.grid(True)

# Set x-axis ticks for every second year
plt.xticks(delays_year_df['year'][::2])

plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Calculate the average and standard deviation for weather_delays_126
avg_delay_126 = delays_year_df.loc[10:, 'weather_delays_126'].mean()
std_delay_126 = delays_year_df.loc[10:, 'weather_delays_126'].std()

# Plotting
plt.figure(figsize=(10, 6))

# Plot historical values
plt.plot(delays_year_df['year'], delays_year_df['weather_delays_historical'], color='gray', linestyle='-', marker='o', label='Historical')

# Plot average lines and spreads for non-NaN values
plt.plot(delays_year_df.loc[10:, 'year'], [avg_delay_126] * len(delays_year_df.loc[10:]), color='r', linestyle='--', label='Avg Delay')
plt.fill_between(delays_year_df.loc[10:, 'year'], avg_delay_126 - std_delay_126, avg_delay_126 + std_delay_126, where=~delays_year_df['weather_delays_126'].isna()[10:], color='lightcoral', alpha=0.5, label='Spread')

# Plot individual data points
plt.scatter(delays_year_df.loc[10:, 'year'], delays_year_df.loc[10:, 'weather_delays_126'], color='darkred', label='Data', marker='o', s=50)

# Plot error bars for individual data points
plt.errorbar(delays_year_df.loc[10:, 'year'], delays_year_df.loc[10:, 'weather_delays_126'], yerr=std_delay_126, linestyle='', color='darkred', label='Std Dev', capsize=4)

plt.xlabel('Year [-]', fontname='Times New Roman', fontsize=16)
plt.ylabel('Number of Delays [-]', fontname='Times New Roman', fontsize=16)
plt.legend(loc='upper left', title=None)
plt.grid(True)

# Set x-axis ticks for every second year
plt.xticks(delays_year_df['year'][::2])

plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Calculate the average and standard deviation for weather_delays_126
avg_cost_126 = cost_year_df.loc[10:, 'Delay Cost 126'].mean()
std_cost_126 = cost_year_df.loc[10:, 'Delay Cost 126'].std()

# Convert cost values to millions
cost_year_df['Delay Cost Historical (Millions)'] = cost_year_df['Delay Cost Historical'] / 1000000
cost_year_df['Delay Cost 126 (Millions)'] = cost_year_df['Delay Cost 126'] / 1000000

# Plotting
plt.figure(figsize=(10, 6))

# Plot historical values
plt.plot(cost_year_df['year'], cost_year_df['Delay Cost Historical (Millions)'], color='gray', linestyle='-', marker='o', label='Historical')

# Plot average line and spread for non-NaN values
plt.plot(cost_year_df.loc[10:, 'year'], [avg_cost_126 / 1000000] * len(cost_year_df.loc[10:]), color='r', linestyle='--', label='Avg Delay Cost')
plt.fill_between(cost_year_df.loc[10:, 'year'], (avg_cost_126 - std_cost_126) / 1000000, (avg_cost_126 + std_cost_126) / 1000000, where=~cost_year_df['Delay Cost 126'].isna()[10:], color='lightcoral', alpha=0.5, label='Spread')

# Plot individual data points
plt.scatter(cost_year_df.loc[10:, 'year'], cost_year_df.loc[10:, 'Delay Cost 126 (Millions)'], color='darkred', label='Data', marker='o', s=50)

# Plot error bars for individual data points
plt.errorbar(cost_year_df.loc[10:, 'year'], cost_year_df.loc[10:, 'Delay Cost 126 (Millions)'], yerr=std_cost_126 / 1000000, linestyle='', color='darkred', label='Std Dev', capsize=4)

plt.xlabel('Year [-]', fontname='Times New Roman', fontsize=16)
plt.ylabel('Cost of Delay [$ Million]', fontname='Times New Roman', fontsize=16)
plt.legend(loc='upper left', title=None)

plt.grid(True)

# Set x-axis ticks for every second year
plt.xticks(cost_year_df['year'][::2])

plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Calculate the average and standard deviation for weather_delays_585
avg_cost_585 = cost_year_df.loc[10:, 'Delay Cost 585'].mean()
std_cost_585 = cost_year_df.loc[10:, 'Delay Cost 585'].std()

# Plotting
plt.figure(figsize=(10, 6))

# Plot historical values
plt.plot(cost_year_df['year'], cost_year_df['Delay Cost Historical'] / 1e6, color='gray', linestyle='-', marker='o', label='Historical')  # Divide by 1e6 to convert to millions

# Plot average lines and spreads for non-NaN values
plt.plot(cost_year_df.loc[10:, 'year'], [avg_cost_585 / 1e6] * len(cost_year_df.loc[10:]), color='b', linestyle='--', label='Avg Delay Cost')  # Divide by 1e6
plt.fill_between(cost_year_df.loc[10:, 'year'], (avg_cost_585 - std_cost_585) / 1e6, (avg_cost_585 + std_cost_585) / 1e6, where=~cost_year_df['Delay Cost 585'].isna()[10:], color='lightblue', alpha=0.5, label='Spread')  # Divide by 1e6

# Plot individual data points
plt.scatter(cost_year_df.loc[10:, 'year'], cost_year_df.loc[10:, 'Delay Cost 585'] / 1e6, color='darkblue', label='Data', marker='o', s=50)  # Divide by 1e6

# Plot error bars for individual data points
plt.errorbar(cost_year_df.loc[10:, 'year'], cost_year_df.loc[10:, 'Delay Cost 585'] / 1e6, yerr=std_cost_585 / 1e6, linestyle='', color='darkblue', label='Std Dev', capsize=4)  # Divide by 1e6

plt.xlabel('Year [-]', fontname='Times New Roman', fontsize=16)
plt.ylabel('Cost of Delay [$ Million]', fontname='Times New Roman', fontsize=16)  # Updated ylabel
plt.legend(loc='upper left', title=None)
plt.grid(True)

# Set x-axis ticks for every second year
plt.xticks(cost_year_df['year'][::2])

plt.show()


In [ ]:
# calculate kendall tau and spearman rank correlation for weather delays results

column_126 = delays_year_df['weather_delays_126']
column_585 = delays_year_df['weather_delays_585']

# Drop NaN values from both year and delay columns
valid_data = delays_year_df.dropna(subset=['year', 'weather_delays_126', 'weather_delays_585'])

# Extract valid columns
valid_year = valid_data['year']
valid_126 = valid_data['weather_delays_126']
valid_585 = valid_data['weather_delays_585']

# Calculate Kendall tau and p-value
tau_126, p_value_126 = kendalltau(valid_year, valid_126)

# Calculate Spearman rank correlation and p-value
rho_126, p_value_126_spearman = spearmanr(valid_year, valid_126)

print(f"Kendall tau for weather_delays_126: {tau_126}, p-value: {p_value_126}")
print(f"Spearman rank correlation for weather_delays_126: {rho_126}, p-value: {p_value_126_spearman}")

# Repeat the process for weather_delays_585
tau_585, p_value_585 = kendalltau(valid_year, valid_585)
rho_585, p_value_585_spearman = spearmanr(valid_year, valid_585)

print(f"Kendall tau for weather_delays_585: {tau_585}, p-value: {p_value_585}")
print(f"Spearman rank correlation for weather_delays_585: {rho_585}, p-value: {p_value_585_spearman}")


In [ ]:
# PLotting the cost per carrier

In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2023 = corrected_ssp126_df_2023.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2023['Average_Delay_Cost_Per_Flight'] = grouped_data_2023['sum'] / grouped_data_2023['count']

# Create a new DataFrame with the result
result_df_2023 = grouped_data_2023.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2023)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2024 = corrected_ssp126_df_2024.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2024['Average_Delay_Cost_Per_Flight'] = grouped_data_2024['sum'] / grouped_data_2024['count']

# Create a new DataFrame with the result
result_df_2024 = grouped_data_2024.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2024)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2025 = corrected_ssp126_df_2025.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2025['Average_Delay_Cost_Per_Flight'] = grouped_data_2025['sum'] / grouped_data_2025['count']

# Create a new DataFrame with the result
result_df_2025 = grouped_data_2025.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2025)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2026 = corrected_ssp126_df_2026.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2026['Average_Delay_Cost_Per_Flight'] = grouped_data_2026['sum'] / grouped_data_2026['count']

# Create a new DataFrame with the result
result_df_2026 = grouped_data_2026.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2026)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2027 = corrected_ssp126_df_2027.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2027['Average_Delay_Cost_Per_Flight'] = grouped_data_2027['sum'] / grouped_data_2027['count']

# Create a new DataFrame with the result
result_df_2027 = grouped_data_2027.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2027)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2028 = corrected_ssp126_df_2028.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2028['Average_Delay_Cost_Per_Flight'] = grouped_data_2028['sum'] / grouped_data_2028['count']

# Create a new DataFrame with the result
result_df_2028 = grouped_data_2028.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2028)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2029 = corrected_ssp126_df_2029.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2029['Average_Delay_Cost_Per_Flight'] = grouped_data_2029['sum'] / grouped_data_2029['count']

# Create a new DataFrame with the result
result_df_2029 = grouped_data_2029.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2029)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2030 = corrected_ssp126_df_2030.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2030['Average_Delay_Cost_Per_Flight'] = grouped_data_2030['sum'] / grouped_data_2030['count']

# Create a new DataFrame with the result
result_df_2030 = grouped_data_2030.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2030)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2030 = corrected_ssp126_df_2030.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2030['Average_Delay_Cost_Per_Flight'] = grouped_data_2030['sum'] / grouped_data_2030['count']

# Create a new DataFrame with the result
result_df_2030 = grouped_data_2030.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2030)


In [ ]:
dataframes = [result_df_2023, result_df_2024, result_df_2025, result_df_2026, result_df_2027, result_df_2028, result_df_2029, result_df_2030]

# Concatenate all DataFrames along the rows
merged_df = pd.concat(dataframes, ignore_index=True)

# Group by 'OP_UNIQUE_CARRIER' and calculate the mean for 'Average_Delay_Cost_Per_Flight'
average_delay_by_carrier_SSP126 = merged_df.groupby('OP_UNIQUE_CARRIER')['Average_Delay_Cost_Per_Flight'].mean().reset_index()

# Print the resulting DataFrame
print(average_delay_by_carrier_SSP126)


In [ ]:
dataframes = [result_df_2023, result_df_2024, result_df_2025, result_df_2026, result_df_2027, result_df_2028, result_df_2029, result_df_2030]

# Concatenate all DataFrames along the rows
merged_df = pd.concat(dataframes, ignore_index=True)

# Group by 'OP_UNIQUE_CARRIER' and calculate the mean for 'Average_Delay_Cost_Per_Flight'
average_delay_overall_SSP126 = merged_df['Average_Delay_Cost_Per_Flight'].mean()

# Print the resulting DataFrame
print(average_delay_overall_SSP126)


In [ ]:
# 585

In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2023_585 = corrected_ssp585_df_2023.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2023_585['Average_Delay_Cost_Per_Flight'] = grouped_data_2023_585['sum'] / grouped_data_2023_585['count']

# Create a new DataFrame with the result
result_df_2023_585 = grouped_data_2023_585.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2023_585)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2024_585 = corrected_ssp585_df_2024.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2024_585['Average_Delay_Cost_Per_Flight'] = grouped_data_2024_585['sum'] / grouped_data_2024_585['count']

# Create a new DataFrame with the result
result_df_2024_585 = grouped_data_2024_585.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2024_585)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2025_585 = corrected_ssp585_df_2025.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2025_585['Average_Delay_Cost_Per_Flight'] = grouped_data_2025_585['sum'] / grouped_data_2025_585['count']

# Create a new DataFrame with the result
result_df_2025_585 = grouped_data_2025_585.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2025_585)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2026_585 = corrected_ssp585_df_2026.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2026_585['Average_Delay_Cost_Per_Flight'] = grouped_data_2026_585['sum'] / grouped_data_2026_585['count']

# Create a new DataFrame with the result
result_df_2026_585 = grouped_data_2026_585.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2026_585)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2027_585 = corrected_ssp585_df_2027.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2027_585['Average_Delay_Cost_Per_Flight'] = grouped_data_2027_585['sum'] / grouped_data_2027_585['count']

# Create a new DataFrame with the result
result_df_2027_585 = grouped_data_2027_585.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2027_585)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2028_585 = corrected_ssp585_df_2028.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2028_585['Average_Delay_Cost_Per_Flight'] = grouped_data_2028_585['sum'] / grouped_data_2028_585['count']

# Create a new DataFrame with the result
result_df_2028_585 = grouped_data_2028_585.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2028_585)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2029_585 = corrected_ssp585_df_2029.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2029_585['Average_Delay_Cost_Per_Flight'] = grouped_data_2029_585['sum'] / grouped_data_2029_585['count']

# Create a new DataFrame with the result
result_df_2029_585 = grouped_data_2029_585.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2029_585)


In [ ]:
# Group by 'OP_UNIQUE_CARRIER' and calculate the sum of 'Predicted_Delay_Cost' and count of occurrences
grouped_data_2030_585 = corrected_ssp585_df_2030.groupby('OP_UNIQUE_CARRIER')['Predicted_Delay_Cost'].agg(['sum', 'count'])

# Calculate the average delay cost per flight
grouped_data_2030_585['Average_Delay_Cost_Per_Flight'] = grouped_data_2030_585['sum'] / grouped_data_2030_585['count']

# Create a new DataFrame with the result
result_df_2030_585 = grouped_data_2030_585.reset_index()[['OP_UNIQUE_CARRIER', 'Average_Delay_Cost_Per_Flight']]

# Display or use the result_df as needed
print(result_df_2030_585)


In [ ]:
dataframes_585 = [result_df_2023_585, result_df_2024_585, result_df_2025_585, result_df_2026_585, result_df_2027_585, result_df_2028_585, result_df_2029_585, result_df_2030_585]

# Concatenate all DataFrames along the rows
merged_df_585 = pd.concat(dataframes_585, ignore_index=True)

# Group by 'OP_UNIQUE_CARRIER' and calculate the mean for 'Average_Delay_Cost_Per_Flight'
average_delay_by_carrier_SSP585 = merged_df_585.groupby('OP_UNIQUE_CARRIER')['Average_Delay_Cost_Per_Flight'].mean().reset_index()

# Print the resulting DataFrame
print(average_delay_by_carrier_SSP585)


In [ ]:
dataframes_585 = [result_df_2023_585, result_df_2024_585, result_df_2025_585, result_df_2026_585, result_df_2027_585, result_df_2028_585, result_df_2029_585, result_df_2030_585]

# Concatenate all DataFrames along the rows
merged_df_585 = pd.concat(dataframes_585, ignore_index=True)

# Calculate the mean for 'Average_Delay_Cost_Per_Flight' across all rows
average_delay_overall_SSP585 = merged_df_585['Average_Delay_Cost_Per_Flight'].mean()

# Print the resulting mean value
print("Overall Average Delay Cost Per Flight (SSP585):", average_delay_overall_SSP585)


In [ ]:
# Calculate by what percentage pices need to be increased to cover the average delay cost - SSP 5.85
297.96 /  416.17 *100

In [ ]:
# Calculate by what percentage pices need to be increased to cover the average delay cost - SSP 1.26
304.78 / 416.17*100

In [ ]:
# PLot the average Delay Cost Per Flight by Carrier (SSP126)

# Sort the DataFrame by 'Average_Delay_Cost_Per_Flight' in ascending order
average_delay_by_carrier_SSP126_sorted = average_delay_by_carrier_SSP126.sort_values(by='Average_Delay_Cost_Per_Flight')

# Set the color palette for the plot
colors = sns.color_palette("coolwarm", len(average_delay_by_carrier_SSP126_sorted))

# Plot the bar plot
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='Average_Delay_Cost_Per_Flight', y='OP_UNIQUE_CARRIER', data=average_delay_by_carrier_SSP126_sorted, palette=colors)

# Add values on top of the bars
for p in ax.patches:
    ax.annotate(f'{p.get_width():.2f}', (p.get_x() + p.get_width() / 2., p.get_y() + p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# Set labels and title
plt.xlabel('Average Delay Cost Per Flight [$]')
plt.ylabel('Operating Carrier')
#plt.title('Average Delay Cost Per Flight by Carrier (SSP126)')

# Show the plot
plt.show()


In [ ]:
# PLot the average Delay Cost Per Flight by Carrier (SSP585)

# Sort the DataFrame by 'Average_Delay_Cost_Per_Flight' in ascending order
average_delay_by_carrier_SSP585_sorted = average_delay_by_carrier_SSP585.sort_values(by='Average_Delay_Cost_Per_Flight')

# Set the color palette for the plot
colors = sns.color_palette("coolwarm", len(average_delay_by_carrier_SSP585_sorted))

# Plot the bar plot
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='Average_Delay_Cost_Per_Flight', y='OP_UNIQUE_CARRIER', data=average_delay_by_carrier_SSP585_sorted, palette=colors)

# Add values on top of the bars
for p in ax.patches:
    ax.annotate(f'{p.get_width():.2f}', (p.get_x() + p.get_width() / 2., p.get_y() + p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# Set labels and title
plt.xlabel('Average Delay Cost Per Flight [$]')
plt.ylabel('Operating Carrier')

# Show the plot
plt.show()


In [ ]:
# Comparing delay percentage of departure times

In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp126_df_2023['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp126_df_2023['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_126_23 = corrected_ssp126_df_2023.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_126_23['year'] = 2023

avg_delays_per_hour_126_23['data'] = '126'

# Display the resulting DataFrame
print(avg_delays_per_hour_126_23)

In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp126_df_2024['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp126_df_2024['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_126_24 = corrected_ssp126_df_2024.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_126_24['year'] = 2024

avg_delays_per_hour_126_24['data'] = '126'

# Display the resulting DataFrame
print(avg_delays_per_hour_126_24)


In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp126_df_2025['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp126_df_2025['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_126_25 = corrected_ssp126_df_2025.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_126_25['year'] = 2025

avg_delays_per_hour_126_25['data'] = '126'

# Display the resulting DataFrame
print(avg_delays_per_hour_126_25)

In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp126_df_2026['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp126_df_2026['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_126_26 = corrected_ssp126_df_2026.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_126_26['year'] = 2026

avg_delays_per_hour_126_26['data'] = '126'

# Display the resulting DataFrame
print(avg_delays_per_hour_126_26)

In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp126_df_2027['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp126_df_2027['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_126_27 = corrected_ssp126_df_2027.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_126_27['year'] = 2027

avg_delays_per_hour_126_27['data'] = '126'

# Display the resulting DataFrame
print(avg_delays_per_hour_126_27)

In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp126_df_2028['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp126_df_2028['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_126_28 = corrected_ssp126_df_2028.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_126_28['year'] = 2028

avg_delays_per_hour_126_28['data'] = '126'

# Display the resulting DataFrame
print(avg_delays_per_hour_126_28)

In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp126_df_2029['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp126_df_2029['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_126_29 = corrected_ssp126_df_2029.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_126_29['year'] = 2029

avg_delays_per_hour_126_29['data'] = '126'

# Display the resulting DataFrame
print(avg_delays_per_hour_126_29)

In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp126_df_2030['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp126_df_2030['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_126_30 = corrected_ssp126_df_2030.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_126_30['year'] = 2030

avg_delays_per_hour_126_30['data'] = '126'

# Display the resulting DataFrame
print(avg_delays_per_hour_126_30)

In [ ]:
# create a list containing your DataFrames
dfs = [avg_delays_per_hour_126_23, avg_delays_per_hour_126_24, avg_delays_per_hour_126_25,
       avg_delays_per_hour_126_26, avg_delays_per_hour_126_27, avg_delays_per_hour_126_28,
       avg_delays_per_hour_126_29, avg_delays_per_hour_126_30]

# Concatenate all DataFrames into one
df_concatenated = pd.concat(dfs)

# Group by CRS_DEP_TIME and calculate the mean of Weather_Delayed
result_df = df_concatenated.groupby('CRS_DEP_TIME')['Weather_Delayed'].mean().reset_index()

# Display the result
print(result_df)

In [ ]:
# result_df is the DataFrame with average Weather_Delayed values
result_df['Weather_Delayed_percentage'] = result_df['Weather_Delayed'] * 100

plt.figure(figsize=(10, 6))

# Plot the line and fill the area under the curve
plt.fill_between(result_df['CRS_DEP_TIME'], result_df['Weather_Delayed_percentage'], color='skyblue', alpha=0.4)
plt.plot(result_df['CRS_DEP_TIME'], result_df['Weather_Delayed_percentage'], linestyle='-', color='b')

#plt.title('Percentage of Delayed Flights Over Time', fontdict={'fontname': 'Times New Roman', 'fontsize': 16})
plt.xlabel('Departure Time [Hour of the Day]', fontdict={'fontname': 'Times New Roman', 'fontsize': 14})
plt.ylabel('Percentage of Delayed Flights [%]', fontdict={'fontname': 'Times New Roman', 'fontsize': 14})
plt.ylim(0, 100)
plt.grid(True)
plt.show()


In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp585_df_2023['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp585_df_2023['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_585_23 = corrected_ssp585_df_2023.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_585_23['year'] = 2023

avg_delays_per_hour_585_23['data'] = '585'

# Display the resulting DataFrame
print(avg_delays_per_hour_585_23)


In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp585_df_2024['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp585_df_2024['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_585_24 = corrected_ssp585_df_2024.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_585_24['year'] = 2024

avg_delays_per_hour_585_24['data'] = '585'

# Display the resulting DataFrame
print(avg_delays_per_hour_585_24)


In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp585_df_2025['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp585_df_2025['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_585_25 = corrected_ssp585_df_2025.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_585_25['year'] = 2025

avg_delays_per_hour_585_25['data'] = '585'

# Display the resulting DataFrame
print(avg_delays_per_hour_585_25)


In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp585_df_2026['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp585_df_2026['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_585_26 = corrected_ssp585_df_2026.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_585_26['year'] = 2026

avg_delays_per_hour_585_26['data'] = '585'

# Display the resulting DataFrame
print(avg_delays_per_hour_585_26)


In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp585_df_2027['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp585_df_2027['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_585_27 = corrected_ssp585_df_2027.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_585_27['year'] = 2027

avg_delays_per_hour_585_27['data'] = '585'

# Display the resulting DataFrame
print(avg_delays_per_hour_585_27)


In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp585_df_2028['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp585_df_2028['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_585_28 = corrected_ssp585_df_2028.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_585_28['year'] = 2028

avg_delays_per_hour_585_28['data'] = '585'

# Display the resulting DataFrame
print(avg_delays_per_hour_585_28)


In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp585_df_2029['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp585_df_2029['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_585_29 = corrected_ssp585_df_2029.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_585_29['year'] = 2029

avg_delays_per_hour_585_29['data'] = '585'

# Display the resulting DataFrame
print(avg_delays_per_hour_585_29)


In [ ]:
# Convert CRS_DEP_TIME to datetime format
corrected_ssp585_df_2030['CRS_DEP_TIME'] = pd.to_datetime(corrected_ssp585_df_2030['CRS_DEP_TIME'], format='%H%M').dt.hour

# Group by departure time and calculate average delays
avg_delays_per_hour_585_30 = corrected_ssp585_df_2030.groupby('CRS_DEP_TIME')['Weather_Delayed', 'Predicted_Delay_Cost'].mean().reset_index()

avg_delays_per_hour_585_30['year'] = 2030

avg_delays_per_hour_585_30['data'] = '585'

# Display the resulting DataFrame
print(avg_delays_per_hour_585_30)


In [ ]:
# create a list containing your DataFrames
dfs = [avg_delays_per_hour_585_23, avg_delays_per_hour_585_24, avg_delays_per_hour_585_25,
       avg_delays_per_hour_585_26, avg_delays_per_hour_585_27, avg_delays_per_hour_585_28,
       avg_delays_per_hour_585_29, avg_delays_per_hour_585_30]

# Concatenate all DataFrames into one
df_concatenated = pd.concat(dfs)

# Group by CRS_DEP_TIME and calculate the mean of Weather_Delayed
result_df = df_concatenated.groupby('CRS_DEP_TIME')['Weather_Delayed'].mean().reset_index()

# Display the result
print(result_df)


In [ ]:
# Assuming result_df is the DataFrame with average Weather_Delayed values
result_df['Weather_Delayed_percentage'] = result_df['Weather_Delayed'] * 100

plt.figure(figsize=(10, 6))

# Plot the line and fill the area under the curve
plt.fill_between(result_df['CRS_DEP_TIME'], result_df['Weather_Delayed_percentage'], color='skyblue', alpha=0.4)
plt.plot(result_df['CRS_DEP_TIME'], result_df['Weather_Delayed_percentage'], linestyle='-', color='b')

#plt.title('Percentage of Delayed Flights Over Time', fontdict={'fontname': 'Times New Roman', 'fontsize': 16})
plt.xlabel('Departure Time [Hour of the Day]', fontdict={'fontname': 'Times New Roman', 'fontsize': 14})
plt.ylabel('Percentage of Delayed Flights [%]', fontdict={'fontname': 'Times New Roman', 'fontsize': 14})
plt.ylim(0, 100)
plt.grid(True)
plt.show()


In [ ]:
# Formulas

In [ ]:
# Profit formula

$\text{Profit} = \text{Revenue} - \text{Cost}$


In [ ]:
# Creation of weather_delayed variable formula

\[ \text{Profit} = \text{Revenue} - \text{Cost} \]

$\text{Profit} = \text{Selling Price} \times \text{Quantity Sold} - \text{Cost}$

\begin{cases}
1 & \text{if } \text{WEATHER_DELAY} > 0 \text{ and/or } \text{NAS_DELAY} > 0 \\
0 & \text{otherwise}
\end{cases}